In [1]:
import pandas as pd
from konlpy.tag import Komoran
from konlpy.tag import Okt
from pykospacing import spacing
import re

In [2]:
komoran = Komoran(userdic="./komoran/user_dic.txt")

In [3]:
print(komoran.modelpath)
print(komoran.jki)

C:\Users\gekri\anaconda3\envs\AI_dev_env\lib\site-packages\konlpy\java\data\models
kr.co.shineware.nlp.komoran.core.Komoran@7e0b37bc


In [5]:
print(komoran.pos('수상한 작가 로베르토 아를트의 대표작 『7인의 미치광이』의 완역본. '))
make_keywords('수상한 작가 로베르토 아를트의 대표작 『7인의 미치광이』의 완역본. ')

[('수상', 'XR'), ('하', 'XSA'), ('ㄴ', 'ETM'), ('작가', 'NNG'), ('로베르토', 'NNP'), ('아를트의', 'NA'), ('대표작', 'NNG'), ('『', 'SS'), ('7', 'SN'), ('인', 'NNG'), ('의', 'JKG'), ('미치광이', 'NNG'), ('』', 'SS'), ('의', 'JKG'), ('완역본', 'NNG'), ('.', 'SF')]
수상한 작가 로베르토 아를트의 대표작 '7인의 미치광이'의 완역본. 
수상 작가 로베르토 아를트의 대표작 인 미치광이 완역본


In [68]:
print(komoran.pos('인, 의, 예'))
print(komoran.pos('20인이 있었다'))
print(komoran.pos("존 그린의 주특기인 알싸한 로맨스"))

[('인', 'NNG'), (',', 'SP'), ('의', 'NNG'), (',', 'SP'), ('예', 'NNG')]
[('20', 'SN'), ('인', 'NNG'), ('이', 'JKS'), ('있', 'VX'), ('었', 'EP'), ('다', 'EC')]
[('존 그린', 'NNP'), ('의', 'JKG'), ('주특기', 'NNG'), ('이', 'VCP'), ('ㄴ', 'ETM'), ('알싸', 'XR'), ('하', 'XSA'), ('ㄴ', 'ETM'), ('로맨스', 'NNP')]


In [76]:
print(komoran.pos('지난하다'))
print(komoran.pos('지난일은 신경쓰지마'))

[('지난', 'XR'), ('하', 'XSA'), ('다', 'EC')]
[('지나', 'VV'), ('ㄴ', 'ETM'), ('일', 'NNP'), ('은', 'JX'), ('신경', 'NNG'), ('쓰', 'VV'), ('지', 'EC'), ('말', 'VX'), ('아', 'EC')]


In [89]:
print(komoran.pos('그는 정이 없다.'))
print(komoran.pos('그의 여행을 그대로 따라했을 정도다.'))

[('그', 'NP'), ('는', 'JX'), ('정', 'NNG'), ('이', 'JKS'), ('없', 'VA'), ('다', 'EF'), ('.', 'SF')]
[('그', 'NP'), ('의', 'JKG'), ('여행', 'NNG'), ('을', 'JKO'), ('그대로', 'MAG'), ('따르', 'VV'), ('아', 'EC'), ('하', 'VX'), ('았', 'EP'), ('을', 'ETM'), ('정도', 'NNG'), ('다', 'JX'), ('.', 'SF')]


In [93]:
print(komoran.pos('대상 최우수상 우수상'))

[('대상', 'NNG'), ('최우', 'NNP'), ('수상', 'NNP'), ('우수', 'NNG'), ('상', 'XSN')]


In [4]:
TAG_NORMAL = {'NA', 'NF', 'NNG', 'NNP', 'XR'}

PAT_NO_KOREAN = re.compile(r'[^가-힣]*')
PAT_QUOTES = re.compile(r'[󰡔󰡕󰡒󰡓\[\]「」『』<>〈〉《》‘’“”]')

PAT_ADDITIONAL = re.compile(r'\([^가-힣]+\)')
PAT_NO_MEANING = re.compile(r'[^가-힣0-9a-zA-Z.,"\'() ]')
PAT_PAREN = re.compile(r'[()]')
PAT_OTHERS = re.compile(r'[.,"\']')

def make_keywords(text):
    text = re.sub('\s', ' ', text)
    
    # 한글, 숫자, 문장부호 외 제거
    text = PAT_QUOTES.sub("'", text)     # 특수 기호 조정
    text = PAT_ADDITIONAL.sub('', text)  # 한글 없는 () 제거
    text = PAT_NO_MEANING.sub(' ', text)
    print(text)

    # tokenizing
    token = komoran.pos(text)
    df_token = pd.DataFrame(token, columns=['word', 'tag'])

    # token 정제
    keywords = df_token[df_token['tag'].isin(TAG_NORMAL)]['word']
    
    # 정제되지 않은 숫자와 단위 제거. 단, 문장의 일부인 경우는 수용
    keywords = keywords[keywords.str.match('[^0-9]|([0-9].+ [가-힣])')]
    
    cleaned_keywords = []
    for word in keywords:
        # token의 있는 ' ' 제거 (' '가 구분자라서 ' '이 있는 경우 하나의 token으로 인식하지 못하기 때문에)
        word = word.replace(' ', '')
        
        # 괄호가 있는 경우 두 단어가 분리되지 않은 형태로 판단
        word = word.replace('(', ' ')
        loc = word.find(')')
        if loc > -1:
            word = word[:loc]
        
        # 남은 문장부호들 제거
        word = PAT_OTHERS.sub('', word)
        
        cleaned_keywords.append(word)
        
    keyword_sentence = ' '.join(cleaned_keywords)
    print(keyword_sentence)

In [100]:
spacing("""부재하는 현존으로서 연결되는 마일스,
‘데어’ ‘벗’ ‘포’ ‘더’ 네 가지의 색다른 이야기

한 남자가 있었다. 그는 우연히 초대된 디너파티에서 디저트를 기다리던 사이 위층에 올라가
더니 예비 침실로 들어 가 문을 잠근다. 그대로 며칠, 몇 주, 몇 달이 지나도록 나오지 않는다.
낯선 손님의 엉뚱한 행동에 집주인 제네비브 리는 그의 지인을 수소문하고 언론에 자신의 처
지를 기고하는 등 애를 쓴다. 한편 그의 의도가 무엇이든 상관없이 이 마일스 가스의 이야기
는 언론의 주목을 받아 점점 널리 퍼지고 전국에서 창문을 통해 그의 손이라도 보기 위해 사
람들이 몰려온다.
이 소설은 여러 모로 읽는 이를 낯설게 한다. 예컨대 전체 분량의 반 정도가 괄호 안에 쓰였다.
또 이야기의 문을 연 마일스 가스에 관한 이야기가 펼쳐지리라 기대하는데 그는 계속해서 이
야기 속에서 존재하지 않으면서 존재한다. 마일스는 ‘데어’ ‘벗’ ‘포’ ‘더’ 네 가지의 이야기를 통
해 조금씩 암호처럼 드러날 뿐이다. 그는 그렇게 부재하는 현존으로서 모두와 연결될 뿐 아니
라 조용한 변화를 불러일으킨다. 각 장의 제목이 되는 단어들은 해당 이야기 속에서 빈도 높
게 사용되며 스토리의 핵심으로 작용한다.""".replace('\n', ''))

'부재하는 현존으로서 연결되는 마일스, ‘데어’ ‘벗’ ‘포’ ‘더’ 네 가지의 색다른 이야기한 남자가 있었다. 그는 우연히 초대된 디너 파티에서 디저트를 기다리던 사이 위층에 올라가더니 예비 침실로 들어 가 문을 잠근다. 그대로 며칠, 몇 주, 몇 달이 지나도록 나오지 않는다. 낯선 손님의 엉뚱한 행동에 집주인 제네비브 리는 그의 지인을 수소문하고 언론에 자 신의 처지를 기고 하는 등 애를 쓴다. 한 편 그의 의도가 무엇이든 상관없이 이 마일스 가스의 이야기는 언론의 주목을 받아 점점 널리 퍼지고 전국에서 창문을 통해 그의 손이라도 보기 위해 사람들이 몰려온다.이 소설은 여러 모로 읽는 이를 낯설게 한다. 예컨대 전체 분량의 반 정도가 괄호 안에 쓰였다. 또 이야기의 문을 연 마일스 가스에 관한 이야기가 펼쳐지리 라 기 대하는데 그는 계속해서 이야기 속에서 존재하지 않으면서 존재한다. 마일스는 ‘데어’ ‘벗’ ‘포’ ‘더’ 네 가지의 이야기를 통해 조금씩 암호처럼 드러날 뿐이다. 그는 그렇게 부재하는 현존으로서 모두와 연결될 뿐 아니라 조용한 변화를 불러일으킨다. 각 장의 제목이 되는 단어들은 해당 이야기 속에서 빈도 높게 사용되며 스토리의 핵심으로 작용한다.'

In [97]:
make_keywords("""부재하는 현존으로서 연결되는 마일스,
‘데어’ ‘벗’ ‘포’ ‘더’ 네 가지의 색다른 이야기

한 남자가 있었다. 그는 우연히 초대된 디너파티에서 디저트를 기다리던 사이 위층에 올라가
더니 예비 침실로 들어 가 문을 잠근다. 그대로 며칠, 몇 주, 몇 달이 지나도록 나오지 않는다.
낯선 손님의 엉뚱한 행동에 집주인 제네비브 리는 그의 지인을 수소문하고 언론에 자신의 처
지를 기고하는 등 애를 쓴다. 한편 그의 의도가 무엇이든 상관없이 이 마일스 가스의 이야기
는 언론의 주목을 받아 점점 널리 퍼지고 전국에서 창문을 통해 그의 손이라도 보기 위해 사
람들이 몰려온다.
이 소설은 여러 모로 읽는 이를 낯설게 한다. 예컨대 전체 분량의 반 정도가 괄호 안에 쓰였다.
또 이야기의 문을 연 마일스 가스에 관한 이야기가 펼쳐지리라 기대하는데 그는 계속해서 이
야기 속에서 존재하지 않으면서 존재한다. 마일스는 ‘데어’ ‘벗’ ‘포’ ‘더’ 네 가지의 이야기를 통
해 조금씩 암호처럼 드러날 뿐이다. 그는 그렇게 부재하는 현존으로서 모두와 연결될 뿐 아니
라 조용한 변화를 불러일으킨다. 각 장의 제목이 되는 단어들은 해당 이야기 속에서 빈도 높
게 사용되며 스토리의 핵심으로 작용한다.""")

부재하는 현존으로서 연결되는 마일스, '데어' '벗' '포' '더' 네 가지의 색다른 이야기  한 남자가 있었다. 그는 우연히 초대된 디너파티에서 디저트를 기다리던 사이 위층에 올라가 더니 예비 침실로 들어 가 문을 잠근다. 그대로 며칠, 몇 주, 몇 달이 지나도록 나오지 않는다. 낯선 손님의 엉뚱한 행동에 집주인 제네비브 리는 그의 지인을 수소문하고 언론에 자신의 처 지를 기고하는 등 애를 쓴다. 한편 그의 의도가 무엇이든 상관없이 이 마일스 가스의 이야기 는 언론의 주목을 받아 점점 널리 퍼지고 전국에서 창문을 통해 그의 손이라도 보기 위해 사 람들이 몰려온다. 이 소설은 여러 모로 읽는 이를 낯설게 한다. 예컨대 전체 분량의 반 정도가 괄호 안에 쓰였다. 또 이야기의 문을 연 마일스 가스에 관한 이야기가 펼쳐지리라 기대하는데 그는 계속해서 이 야기 속에서 존재하지 않으면서 존재한다. 마일스는 '데어' '벗' '포' '더' 네 가지의 이야기를 통 해 조금씩 암호처럼 드러날 뿐이다. 그는 그렇게 부재하는 현존으로서 모두와 연결될 뿐 아니 라 조용한 변화를 불러일으킨다. 각 장의 제목이 되는 단어들은 해당 이야기 속에서 빈도 높 게 사용되며 스토리의 핵심으로 작용한다.
부재 현존 연결 마일스 데어 벗 포 이야기 남자 초대 디너 파티 디저트 사이 위층 예비 침실 문 며칠 주 달 낯선 손님 엉뚱 행동 집주인 제네비브 지인 수소문 언론 자신 처 지 기고 애 한편 의도 마일스 가스 이야기 언론 주목 전국 창문 손 사 이 소설 모로 전체 분량 반 정도 괄호 안 이야기 문 마일스 가스 이야기 기대 계속 야기 속 존재 존재 마일 데어 벗 포 이야기 조금 암호 부재 현존 모두 연결 라 조용 변화 장의 제목 단어 해당 이야기 속 빈도 높 게 사용 스토리 핵심 작용


In [90]:
make_keywords("""<당나귀와 떠난 여행>은 로버트 루이스 스티븐슨이 ‘모데스틴’이라는 당나귀와 함께 종교분쟁(가톨릭과 프로테스탄트 분쟁)의 한 중심지였던 프랑스 남부의 세벤느를 여행하고 쓴 글이다. 르 퓌로부터 출발하여 120마일이나 되는 험준한 산길을 걸어 생 장 뒤 갸르에 도착하기까지의 과정이 생생하게 살아 있다. 1879년에 발표된 이 여행기는 여행기 장르를 개척한 선구적인 작품으로 손꼽혀 많은 사람들이 세벤느에 와서 그의 여행을 그대로 따라했을 정도다. 1978년에는 스티븐슨의 세벤느 여행 백주년 기념행사가 열려 세계의 많은 RLS(로버트 루이스 스티븐슨의 약칭) 애호가들이 몰려와 당나귀를 끌고 스티븐슨의 여행을 다시 재연하기도 하였다.

이 작품은 참으로 다양한 빛깔을 지니고 있다. 당나귀와의 실랑이를 포함해 도보여행중에 겪는 수많은 에피소드에서는 가벼운 웃음을, 수도원 <눈의 여인>에서는 경건함과 관용을, 종교분쟁지인 세벤느에서는 역사의 슬픔을, 소나무 숲에서 보낸 하룻밤에는 무한한 환희와 낭만을, 당나귀와의 이별 장면에서는 시큰함과 애틋함을 함께 체험하도록 이끈다.

스티븐슨은 자연의 숭고하고 위대한 힘, 이를테면 새벽이 부르는 소리, 쏟아질 듯이 하늘을 가득 채운 별들, 바람이 일으키는 천연의 환기 작용 등을 통해 문명의 혜택을 훌훌 벗어 던지고 잠시 자연 속으로 뛰어들라고 유혹한다. 그런가 하면 자연은 숙박비도 받지 않으면서 아름다운 천장과 천연의 양탄자를 제공하는 최고급의 여인숙이라고도 말한다.

자연 한가운데서 고독을 즐기면서 인간의 삶을 명상하는 스티븐슨의 여행기는, 우리 모두 최소한의 경비만 가지고 배낭을 하나씩 짊어진 채, 황량하고 거칠지만 소박한 우리 시골 한가운데를 향해 떠나고 싶은 마음이 들게 한다.

참고로 원서에는 없는 역자 원유경 교수의 는 작가를 이해하는 좋은 길잡이 역할을 할 것이며, 스티븐슨을 연구하는 데에도 매우 중요한 자료로 쓰일 수 있을 것이다.""")

당나귀 여행 로버트루이스스티븐슨 모 스티 당나귀 종교 분쟁 가톨릭 프로테스탄트 분쟁 중심지 프랑스 남부 벤 느 여행 글 르 퓌로부터 출발 마일 험준 산길 생 장 뒤 갸 르 도착 과정 생생 발표 여행기 여행기 장르 개척 선구 작품 사람 벤 느 여행 정도 스티븐슨 벤 느 여행 기념행사 세계 로버트루이스스티븐슨 약칭 애호가 당나귀 스티븐슨 여행 재연 작품 다양 빛깔 당나귀 실랑이 포함 도보 여행 에피소드 웃음 수도원 눈 여인 경건 관용 종교 분쟁 지인 벤 느 역사 슬픔 소나무 숲 하룻밤 무한 환희 낭만 당나귀 이별 장면 시큰 애틋 체험 스티븐슨 자연 숭고 위대 힘 새벽 소리 하늘 채운 별 바람 천연 환기 작용 문명 혜택 자연 속 유혹 자연 숙박비 천장 천연 양탄자 제공 고급 여인숙 말 자연 한가운데 고독 인간의삶 명상 스티븐슨 여행기 최소한 경비 배낭 황량 소박 시골 한가운데 마음 참고 원서 역자 유경 교수 작가 이해 길잡이 역할 스티븐슨 연구 중요 자료


In [62]:
make_keywords("""SF 판타지의 새로운 지평, N. K. 제미신의 첫 단편집로커스 상·알렉스 상 수상세계환상문학상·영국환상문학상 후보작「부서진 대지」 3부작으로 휴고 상 최우수 장편상을 3년 연속 수상하며 전례 없는 새로운 역사를 쓴 N. K. 제미신의 첫 단편집이 출간되었다. ‘검은 미래의 달까지 얼마나 걸릴까?’는 제목은 저자가 흑인 여성으로서 SF와 판타지를 사랑하기가 얼마나 어려웠는지를 주제로 쓴 동명의 에세이에서 따온 제목으로, 2004년부터 2017년까지 쓰인 22편의 작품이 수록되어 있으며, 로커스 상 최우수 작품집상과 미국도서관협회가 주관하는 알렉스 상을 수상했다. 장편 시리즈를 구상하는 데 바탕이 된 작품(「위대한 도시의 탄생」, 「스톤 헝거」, 「수면 마법사」) 및 SF 거장 어슐러 르 귄과 로버트 하인라인의 걸작에 대한 재해석, 휴고 상·네뷸러 상 최우수 단편상 후보에 올랐던 「비제로 확률」 등 제미신의 폭넓은 작품 세계를 조망할 수 있는 단편들이 담겼다. 비행선이 보편화된 19세기 미국 배경의 스팀펑크물, 23세기 외계 생명체와의 무역 협상 등 그야말로 천차만별의 시공간과 소재를 다루었지만, 다양한 색깔의 인물들 그리고 낡은 질서와 틀에 대한 저항 의식이 작품집 전체를 관통하며 “작가로서, 그리고 운동가로서 성장한 과정을 기록한 연대기”라는 작가의 말을 다시금 되새기게 한다. SF 판타지의 새로운 지평, N. K. 제미신의 첫 단편집스팀펑크, 어반 판타지 등을 망라한 22편의 작품 수록로커스 상·알렉스 상 수상세계환상문학상·영국환상문학상 후보작「부서진 대지」 3부작으로 휴고 상 최우수 장편상을 3년 연속 수상하며 전례 없는 새로운 역사를 쓴 N. K. 제미신의 첫 단편집이 출간되었다. ‘검은 미래의 달까지 얼마나 걸릴까?’는 제목은 저자가 흑인 여성으로서 SF와 판타지를 사랑하기가 얼마나 어려웠는지를 주제로 쓴 동명의 에세이에서 따온 제목으로, 2004년부터 2017년까지 쓰인 22편의 작품이 수록되어 있으며, 로커스 상 최우수 작품집상과 미국도서관협회가 주관하는 알렉스 상을 수상했다. 장편 시리즈를 구상하는 데 바탕이 된 작품(「위대한 도시의 탄생」, 「스톤 헝거」, 「수면 마법사」) 및 SF 거장 어슐러 르 귄과 로버트 하인라인의 걸작에 대한 재해석, 휴고 상·네뷸러 상 최우수 단편상 후보에 올랐던 「비제로 확률」 등 제미신의 폭넓은 작품 세계를 조망할 수 있는 단편들이 담겼다. 비행선이 보편화된 19세기 미국 배경의 스팀펑크물, 23세기 외계 생명체와의 무역 협상 등 그야말로 천차만별의 시공간과 소재를 다루었지만, 다양한 색깔의 인물들 그리고 낡은 질서와 틀에 대한 저항 의식이 작품집 전체를 관통하며 “작가로서, 그리고 운동가로서 성장한 과정을 기록한 연대기”라는 작가의 말을 다시금 되새기게 한다.“그러니 보시라. 저기 미래가 있다. 모두 함께 출발하자.”완벽한 이상 사회부터 인간만이 증발하는 종말까지! 다채로운 사고실험의 향연첫 번째 수록작인 「남아서 싸우는 사람들」은 어슐러 르 귄의 단편 「오멜라스를 떠나는 사람들」의 배경과 비슷하게 행복과 번영이 가득하며, 기술적으로는 훨씬 발전된 도시 ‘움-헬라트’를 무대로 펼쳐진다. 그렇다면 다양성이 존중받고 구성원이 서로를 보살피는 이곳 역시, 오멜라스처럼 지하실의 아이를 희생양 삼아 지탱되고 있을까? 제미신은 도전적이지만 결코 쉽지 않은 선택을 결말에서 보여 준다. 인류의 육체를 지배하는 신체 강탈자를 다룬 로버트 하인라인의 단편 「꼭두각시의 비밀」을 모티브로 삼은 「깨어서 걷기」에서도 마찬가지다.환경 재앙으로 오염된 지구에서 살아가거나 우주로 도피하는 두 가지 선택지 하에서 인류가 분화된 미래를 그린 「용 구름이 뜬 하늘」, 사이버 공간 속의 생명들이 인간 사회로 진출하는 「트로이 소녀」와 「졸업생 대표」 연작, 우주 탐사 중 식민 행성에 무슬림 여성 연구자들만이 살아남는 「천국의 신부들」, 외계 생물에 대한 지적 탐구에서 무역 협상에 이르는 과정을 기록한 「평가자들」, 종교적 도그마에 지배당하는 통제 사회를 배경으로 한 엽편 「엘리베이터 댄서」도 제각기 독특한 상황에 놓인 이색적인 인물들을 인상적인 방식으로 전달한다.종말이라는 테마를 지극히 일상적이고 친근한 방식으로 풀어 나간 작품들도 있다. 「렉스 강가에서」서는 인류가 갑자기 지구상에서 사라진 후에 믿어 줄 신자들이 없어 정처 없이 거리를 누비거나 스타벅스에 들르러 줄을 서는 신과 정령 등의 추상적 존재들이 등장하며, 「너무 많은 어제들, 충분치 못한 내일들」은 채팅이나 이메일 등 온라인상에 남은 기록을 제외한 모든 것이 매일 리셋되는 현실에 놓인 채 타인과의 연결을 갈구하는 개개인을 다룬다. 휴고 상·네뷸러 상 후보작 「비제로 확률」은 지하철 사고 같은 재앙이 빈번히 일어나고 중병이 쉽사리 치유되는, 말하자면 어떤 사건이 발생하는 확률이 역전되어 버린 뉴욕에서 적응해 나가는 소시민의 일상을 그렸다.“하지만 잠깐만. 뒤로 돌아가 보자. 그렇다, 흑인 캐릭터라고 했다.”머리말에서 제미신은 과거에 작가든 작품 내에서든 업계에서 여성과 유색인이 소외당하던 현실을 고하며, 스스로를 제외시킬 수 없었기에 이야기에 꾸준히 흑인 캐릭터를 넣었다고 밝힌다. 수록된 모든 작품에서 그러한 노력이 엿보이지만, 그중에서도 특히 민권운동이 확산되던 1960년대 앨라배마 주를 무대로 사악한 요정에게서 딸을 지키려는 여성의 분투를 다룬 「붉은 흙의 마녀」, 혁명을 통해 노예 제도에서 벗어난 최초의 흑인 공화국인 아이티의 첩자 여성과 미국 혼혈 여성 사이의 로맨스가 그려지는 「폐수 엔진」, 허리케인 카트리나가 닥친 뉴올리언스에서 ‘괴물’이라는 형태로 실체화된 혐오에 대항해 분투하는 인물들을 다룬 「잔잔한 물 아래 도시의 죄인들, 성자들, 용들 그리고 혼령들」은 공고하게 유지되고 있는 인종차별의 현실이 나날이 민낯을 드러내는 지금, 더욱 호소력을 띤 채 강렬한 잔상을 남길 것이다."
""")

판타지 지평 미신 단편집 로커스 상 알렉스 상 수상 세계 환상 문학상 영국 환상 문학상 후보작 대지 휴고 상 최우수 장편 연속 수상 전례 역사 미신 단편집 출간 미래 달 제목 저자 흑인 여성 판타지 사랑 주제 동명 에세이 제목 작품 수록 로커스 상 최우수 작품집 미국 도서관 협회 주관 알렉스 상 수상 장편 시리즈 구상 바탕 작품 위대 도시 탄생 스톤 헝거 수면 마법사 거장 어슐러르귄 로버트 하인 라인 걸작 해석 휴고 상 네뷸러 상 최우수 단편 후보 비제 확률 미신 작품 세계 조망 단편 비행선 보편 미국 배경 스팀펑크 물 외계 생명체 무역 협상 천차만별 시공간 소재 다양 색깔 인물 질서 틀 저항 의식 작품집 전체 관통 작가 운동가 성장 과정 기록 연대기 작가 말 판타지 지평 미신 단편집 스팀펑크 어 반 판타지 망라 작품 수록 로커스 상 알렉스 상 수상 세계 환상 문학상 영국 환상 문학상 후보작 대지 휴고 상 최우수 장편 연속 수상 전례 역사 미신 단편집 출간 미래 달 제목 저자 흑인 여성 판타지 사랑 주제 동명 에세이 제목 작품 수록 로커스 상 최우수 작품집 미국 도서관 협회 주관 알렉스 상 수상 장편 시리즈 구상 바탕 작품 위대 도시 탄생 스톤 헝거 수면 마법사 거장 어슐러르귄 로버트 하인 라인 걸작 해석 휴고 상 네뷸러 상 최우수 단편 후보 비제 확률 미신 작품 세계 조망 단편 비행선 보편 미국 배경 스팀펑크 물 외계 생명체 무역 협상 천차만별 시공간 소재 다양 색깔 인물 질서 틀 저항 의식 작품집 전체 관통 작가 운동가 성장 과정 기록 연대기 작가 말 미래 출발 완벽 이상 사회 인간 증발 종말 다채 사고 실험 향연 수록 작 사람 어슐러르귄 단편 오멜라스를 사람 배경 비슷 행복 번영 기술 발전 도시 움 헬라 무대 다양성 존중 구성원 서로 오멜라스처럼 지하실 아이 희생 양 지탱 미신 도전 선택 결말 인류 육체 지배 신체 강탈 로버트 하인 라인 단편 꼭두각시 비밀 모티브 삼은 걷기 마찬가지 환경 재앙 오염 지구 우주 도피 선택 하 인류 분화 미래 구름 하늘 사이

In [61]:
make_keywords(""""미국의 소설가 E. A, 포의 단편 추리소설. 1843년 발표. 술김에 귀여워하던 검은 고양이를 죽인 남자가 그것과 똑같이 생긴 고양이에 의해서 차차 궁극적인 막달은 골목에 몰리는 상황을 그린 고전풍의 공포 소설이자 그의 대표작의 하나이다."
""")

미국 소설가 포 단편 추리 소설 발표 술김 검은고양이 남자 고양이 차차 궁극 막 달 골목 상황 전풍 공포 소설 대표작


In [60]:
make_keywords(""""옥스퍼드대학교 수학 교수였던 작가 루이스 캐럴은 수줍음 많은 성격의 전형적인 학자였다. 그는 대학 학장이었던 리델의 집을 찾아갔다가 리델의 어린 세 딸을 만났고 그들에게 들려준 이야기가 『이상한 나라의 앨리스』로 탄생했다. 단숨에 당대 최고의 아동문학가로 떠오른 루이스 캐럴은 1871년 『이상한 나라의 앨리스』 속편인 『거울나라의 앨리스』를 발표했다. 『거울나라의 앨리스』는 거울 속 세상으로 들어간 앨리스의 모험을 담은 이야기로, 작가 특유의 재치와 유머가 넘치는 소설이다. 사실 앨리스는 리델의 둘째 딸 이름으로 루이스 캐럴이 평생 사랑했던 뮤즈였다. 루이스 캐럴은 앨리스에게 체스 두는 법을 가르쳐주다가, 체스 게임을 이야기의 도구로 삼아 『거울나라의 앨리스』라는 기발하고 유쾌한 이야기를 만들어냈다.『이상한 나라의 앨리스』가 한여름이 배경이었다면 속편인 『거울나라의 앨리스』는 연말을 며칠 앞둔 1869년 크리스마스가 배경이다. 거울나라는 체스판처럼 생긴 세상으로 붉은 여왕과 하얀 여왕, 삼월 토끼, 모자 장수, 말하는 꽃들, 바다코끼리와 목수, 트위들덤과 트위들디 등이 등장한다. 소설 속 등장인물들은 모두 체스 말과 졸들로 이들에게 이름표를 붙인 다음, 『거울나라의 앨리스』를 펴놓고 게임을 따라 해보면 체스의 남다른 재미를 느낄 수 있다. 또는 자기만의 새로운 게임으로 새 이야기를 만들어도 좋다. 빛나는 작품 ★ 영원한 감동 ‘더클래식 세계문학컬렉션’ 도서출판 더클래식은 일찍이 고전의 가치를 깨닫고 이 시대에 꼭 읽어야 할 작품들을 출간해왔다. ‘더클래식 세계문학컬렉션’은 고전 중에서도 세계적으로 문학적 가치를 인정받고 시대를 뛰어넘어 사랑받는 작품들을 모았다. 고전의 가치는 세월이 흘러도 변하지 않지만 읽는 시대와 사람에 따라 그 의미가 새로워질 수 있다. ‘더클래식 세계문학컬렉션’은 단순히 원문을 있는 그대로 옮기는 번역이 아니라 본래의 원문을 해치지 않으면서도 우리말과 글을 풍부하게 사용하여 원작의 감동을 그대로 전달했다. 고전은 수많은 세월을 거치며 독자에게 다양한 감동과 의미를 깨닫게 해준다. ‘더클래식 세계문학컬렉션’은 오늘을 살아가는 독자들에게 다시 한 번 고전의 의미를 되새기고, 빛나는 고전의 가치를 느끼는 소중한 기회를 제공해줄 것이다.체스판 위에서 펼쳐지는 앨리스의 두 번째 모험옥스퍼드대학교 수학 교수였던 작가 루이스 캐럴은 수줍음 많은 성격의 전형적인 학자였다. 그는 대학 학장이었던 리델의 집을 찾아갔다가 리델의 어린 세 딸을 만났고 그들에게 들려준 이야기가 『이상한 나라의 앨리스』로 탄생했다. 단숨에 당대 최고의 아동문학가로 떠오른 루이스 캐럴은 1871년 『이상한 나라의 앨리스』 속편인 『거울나라의 앨리스』를 발표했다. 『거울나라의 앨리스』는 거울 속 세상으로 들어간 앨리스의 모험을 담은 이야기로, 작가 특유의 재치와 유머가 넘치는 소설이다. 사실 앨리스는 리델의 둘째 딸 이름으로 루이스 캐럴이 평생 사랑했던 뮤즈였다. 루이스 캐럴은 앨리스에게 체스 두는 법을 가르쳐주다가, 체스 게임을 이야기의 도구로 삼아 『거울나라의 앨리스』라는 기발하고 유쾌한 이야기를 만들어냈다. 『이상한 나라의 앨리스』가 한여름이 배경이었다면 속편인 『거울나라의 앨리스』는 연말을 며칠 앞둔 1869년 크리스마스가 배경이다. 거울나라는 체스판처럼 생긴 세상으로 붉은 여왕과 하얀 여왕, 삼월 토끼, 모자 장수, 말하는 꽃들, 바다코끼리와 목수, 트위들덤과 트위들디 등이 등장한다. 소설 속 등장인물들은 모두 체스 말과 졸들로 이들에게 이름표를 붙인 다음, 『거울나라의 앨리스』를 펴놓고 게임을 따라 해보면 체스의 남다른 재미를 느낄 수 있다. 또는 자기만의 새로운 게임으로 새 이야기를 만들어도 좋다. 재기발랄한 언어유희미술에 추상화가가 있다면 루이스 캐럴은 언어의 추상화가라는 말을 들을 정도로, 현실에 존재하지 않는 새로운 형용사들을 만들고 그것으로 시를 지어 소설 속에 집어넣었다. 훗날 프랑스에서 ‘초현실주의의 선구자’라고 칭송했으며 추상적인 언어도 문학이 될 수 있다는 가능성을 열어주었다. 특히 『거울나라의 앨리스』는 영어의 동음이의어와 영국의 전래 동요인 〈마더 구스〉의 압운을 사용하여 기발한 말장난을 많이 한다. 하지만 우리는 영어권이 아니다 보니 재기발랄한 루이스 캐럴의 언어유희를 온전히 즐길 수 없어 아쉬울 뿐이다. 예를 들면, 붉은 여왕이 하얀 여왕을 재우며 부르는 자장가는 ‘rock-a bye baby’라는 〈마더 구스〉의 압운에 가사를 바꿔 부르는 식이다. 그리고 등장인물들도 〈마더 구스〉에 나오는 이들로 영어권 아이들에게는 친숙하다. 앨리스는 거울나라에서 그 인물들을 직접 만나 대화를 나누고 어린아이다운 동심의 세계를 마음껏 맛본다. 그리고 작가가 만들어낸 이상한 언어로 된 시를 읊거나 말장난하는 등장인물들 때문에 혼란스러워한다."
""")

옥스퍼드 대학교 수학 교수 작가 루이스캐럴 수줍음 성격 전형 학자 대학 학장 리델 집 리델 딸 이야기 이상한나라의앨리스 탄생 당대 최고 아동문학 가로 루이스캐럴 이상한나라의앨리스 속편 거울 나라 앨리스 발표 거울 나라 앨리스 거울 속 세상 앨리스 모험 이야기 작가 특유 재치 유머 소설 사실 앨리스 리델 딸 이름 루이스캐럴 평생 사랑 뮤즈 루이스캐럴 앨리스 체스 법 체스 게임 이야기 도구 거울 나라 앨리스 기발 유쾌 이야기 이상한나라의앨리스 한여름 배경 속편 거울 나라 앨리스 연말 며칠 크리스마스 배경 거울 나라 체스판 세상 여왕 여왕 삼월 토끼 모자 장수 말 꽃 바다코끼리 목수 트위들덤과 트위들디 등장 소설 속 등장인물 체스 말 졸 이름표 다음 거울 나라 앨리스 게임 해보면 체스 재미 게임 이야기 작품 영원 감동 더 클래식 세계 문학 컬렉션 도서 출판 더 클래식 고전 가치 시대 작품 출간 더 클래식 세계 문학 컬렉션 고전 세계 문학 가치 시대 사랑 작품 고전 가치 세월 시대 사람 의미 더 클래식 세계 문학 컬렉션 원문 번역 본래 원문 우리말 글 풍부 사용 원작 감동 전달 고전 세월 독자 다양 감동 의미 더 클래식 세계 문학 컬렉션 오늘 독자 고전 의미 고전 가치 소중 기회 제공 체스판 위 앨리스 모험 옥스퍼드 대학교 수학 교수 작가 루이스캐럴 수줍음 성격 전형 학자 대학 학장 리델 집 리델 딸 이야기 이상한나라의앨리스 탄생 당대 최고 아동문학 가로 루이스캐럴 이상한나라의앨리스 속편 거울 나라 앨리스 발표 거울 나라 앨리스 거울 속 세상 앨리스 모험 이야기 작가 특유 재치 유머 소설 사실 앨리스 리델 딸 이름 루이스캐럴 평생 사랑 뮤즈 루이스캐럴 앨리스 체스 법 체스 게임 이야기 도구 거울 나라 앨리스 기발 유쾌 이야기 이상한나라의앨리스 한여름 배경 속편 거울 나라 앨리스 연말 며칠 크리스마스 배경 거울 나라 체스판 세상 여왕 여왕 삼월 토끼 모자 장수 말 꽃 바다코끼리 목수 트위들덤과 트위들디 등장 소설 속 등장인물 체스 말 졸 이름표 다음 거울 나라 앨리스 

In [32]:
make_keywords("""中國 3 大詩人 千詩集
目 次
李 白 詩  333                          次                               7           序                               8 ㅡ 11           題                         12 ㅡ 24           詩                           25  ㅡ 650           畵                                   651  ㅡ   653
杜 甫 詩  333                           次                             656           緖                        657 ㅡ 662           題                        663 ㅡ 677           詩                        678  ㅡ 1232           畵                     1233  ㅡ  1240 
白 樂 天 詩  333                           次                             1243           序                        1244 ㅡ 1249           題                     1250 ㅡ 1263           詩                     1264   ㅡ 1817           畵                     1818  ㅡ   1827 
:
中 國   3 大詩人  千 詩集
1000 詩  總 目錄 
李 白 詩 三百三十 三
 序.春夜宴桃李園                봄날 밤 잔치 도리원 서  
 1.駕去溫泉宮後 贈楊山人         화청궁행가시 양산인 준 노래
 2.江南春懷                    강남 봄 회상 3.江上望?公山                 강 위 환공산 바라봄 
 4.江上吟                      강 위 읊음     
 5.江上秋懷                    강 위 가을 회상 
 6.江夏送?公歸漢東序絶句             강하서  사위 보내고 한동 돌아옴 
 7.江夏行                      강하  읊기  
 8.江行寄遠                      강 멀리 띄우기  
 9.客中行                      여행중 노래
10.去婦詞                      부인 내 버리기 
11.見京兆韋參軍量移東陽二首       위참군 동양유배 보고 2수 
12.見野草中有名白頭翁者               들풀 속에서  백두옹 풀 봄
13.結客少年場行                  협객 소년 시절
14.結襪子                       버선 신은 자   
15.涇川送族弟錞                  경천 물가서 족제 순 보냄
16.經下??橋懷張子房           하비이교 지나며 장자방 회고
17.桂 殿 秋                    계전의 가을
18.?客行                      장사치 노래  
19.高句麗                       고 구 려 
20.古朗月行                     옛 밝은 달 노래
21.枯魚過河泣                    마른 고기 하천 지나며 욺 노래22.古有所思行                    옛 생각 노래23.古 意                         옛  뜻24.古 風"
74028274,白樂天詩 3百33 選集,http://image.yes24.com/goods/74028274/800x0,"백거이(白居易, 772년ㅡ846년) 
자(字)는 낙천(樂天)이고, 號는 취음선생(醉吟先生), 향산거사(香山居士)등으로 불리었다. 
唐나라 때 河南 鄭州 洛陽 부근 新鄭에서 태어났다.
The Life and Times of Po Chu-i 772-846 A.D.
일상적인 언어 구사와 풍자에 뛰어나며, 평이하고 유려한 시풍은 원진(元?)과 함께 원백체(元白體)로 통칭된다. 
작품에 <장한가>, <비파행>이 유명하고, 시문집에 ≪백씨문집≫ 따위가 있다.
백거이는 문학 창작을 삶의 여요락으로 여겼다. 그가 지은 작품의 수는 대략 3,840편이라고 하는데, 문학 작가와 작품의 수가 크게 증가한 중당시대라 하더라도 이같이 많은 작품을 창작했다는 것은 놀라운 일이다. 더구나 그의 작품은 형식이 다양하여 시가에서부터 산문작품에 이르기까지 모든 문학형식을 망라했다. 
백거이는 문학으로써 정치이념을 표현하고 독자의 감정에 호소하여 실제 행동에 옮기도록 하는 것을 문학활동의 목적으로 삼았다. 
백거이는 800년 29세 때 최연소로 진사에 급제해 여러 관직을 거쳤으며, 75세의 나이로 생애를 마감했다.
백거이는 문학 창작을 삶의 보람으로 여겼다. 그가 지은 작품의 수는 대략 3,840편이라고 하는데, 문학 작가와 작품의 수가 크게 증가한 중당시대라 하더라도 이같이 많은 작품을 창작했다는 것은 놀라운 일이다. 더구나 그의 작품은 형식이 다양하여 고체시(古體詩)·금체시(今體詩：율시)·악부(樂府)·가행·부(賦)의 시가에서부터, 지명(誌銘)·제문(祭文)·찬(贊)·기(記)·게(偈)·서(序)·제고(制誥)·조칙·주장(奏狀)·책(策)·판(判)·서간(書簡)의 산문작품에 이르기까지 모든 문학형식을 망라했다.
또한 그는 훌륭한 친구를 많이 사귀었는데, 친구들과 서로 주고 받은 시문에는 친애의 정이 물씬 배어 있다. 특히 원진(元?) 및 유우석(劉禹錫)과의 사이에 오간 글을 모은 〈원백창화집 元白唱和集〉과 〈유백창화집 劉白唱和集〉은 중당시대의 문단을 화려하게 장식한 우정의 결실이라 일컬어진다.
그의 여러 작품 가운데에는 정치이념을 주장한 것도 있고 자신의 감정을 표현한 것도 있는데, 모두 평담한 언어로 알기 쉽게 표현되었으며, 시에 봉급의 액수까지 언급하는 등 매우 당당했다.
때문에 평이하고 속되다는 비판을 받기도 했지만 그것은 비상한 노력과 식견에 의해서 달성된 것이었다. 그는 1편의 시가 완성될 때마다 노파에게 읽어주고 어려워하는 곳을 찾아 고치기까지 할 정도로 퇴고(推敲)를 열심히 했다. 백거이가 자신의 시문에 일상어를 유효적절하게 구사한 것도 그의 표현을 간명하게 한 큰 이유 중의 하나이다.
本,
白 樂 天 詩 333 選集은
백 낙천 대표작 명시 111선으로 다음  백 낙천 대표작 명시 333선집 으로
백 낙천 대표작 명시 333선집 문학세계를  내 문학 안으로  白 樂 天 詩 333 選集가져 와
고대 중국 官, 民, 農 문학세계를  즐거이 白  樂 天 333詩 할 수 있도다 !""")

봄날 밤 잔치 도리 화 청 궁 가시 양산 인 노래 강남 봄 회상 강 위환공 산 강 위 강 위 회상 강 하서 사위 한동 강하 강 여행 노래 부인 위 참구 동양 유배 들풀 속 백두 봄 협객 소년 시절 버선 신 경 천 물가 족제 하비 이교 장자방 회고 계 전 장사치 노래 구 달 노래 고기 하천 욺 노래 생각 노래 뜻 com 백거이 낙천 취음 선생 향 산거 사등 나라 때 부근 TheLife 일상 언어 구사 풍자 평이 유려 시풍 원진 백 통칭 작품 장한가 파행 유명 시문집 백씨 문집 백거이 문학 창작 삶 여 요 락 작품 문학 작가 작품 증가 당 시대 작품 창작 작품 형식 다양 시가 산문 작품 문학 형식 망라 백거이 문학 정치 이념 표현 독자 감정 호소 실제 행동 문학 활동 목적 백거이 때 최 연소 진사 급제 관직 나이 생애 마감 백거이 문학 창작 삶 보람 작품 문학 작가 작품 증가 당 시대 작품 창작 작품 형식 다양 고체시 금 체 율시 악부 가행 부 시가 지명 제문 기 게 제고 조 칙 주장 책 서간 산문 작품 문학 형식 망라 훌륭 친구 친구 시문 친애 정 원진 유우석 사이 글 원 백 창 화집 유백 창 화집 당 시대 문단 화려 장식 우정 결실 작품 가운데 정치 이념 주장 자신 감정 표현 평담 한 언어 표현 봉급 액수 언급 당당 평이 속 비판 비상 노력 식견 달성 시가 완성 때 노파 곳 정도 퇴고 백거이 자신 시문 일상어 유효 적절 구사 표현 간명 이유 은 낙천 대표작 명시 다음 낙천 대표작 명시 선집 낙천 대표작 명시 선집 문학 세계 문학 안 고대 중국 문학 세계


In [33]:
make_keywords("""역사를 거울 삼으면 천하의 흥망성쇠와 미래가 보인다!

중국 최고지도부가 선택한 치세의 지침서 ‘제왕삼부곡’ 완결편
21세기 ‘중국의 꿈’은 세계 최강국이었던 건륭시대의 부활이다

작가 얼웨허는 ‘제왕삼부곡’ 시리즈를 발표하기 시작한 1985년부터 대단원의 막을 내린 1999년까지 20세기의 마지막 15년을 고스란히 강희, 옹정, 건륭 세 황제와 함께 보냈다. 그를 통해 얼웨허는 한자漢字로 무려 5백만 자에 달하는 ‘제왕삼부곡’ 시리즈를 완성함으로써 중국 문학사에 커다란 발자취를 남겼다.

문학적인 측면에서 볼 때 얼웨허의 ‘제왕삼부곡’은 갈수록 성숙한 세련미를 자랑하는가 싶더니 《건륭황제》에 이르러 완연히 농익은 향기를 발산한다. 소설 속 인물의 말을 빌리자면 강희는 창세지조創世之祖이고, 옹정은 입국지조立國之祖이며, 건륭은 개업지주開業之主이다. 그렇듯이 《건륭황제》 역시 얼웨허의 ‘제왕삼부곡’ 중에서도 작가가 가장 심혈을 쏟은 역작이다.
 인류 역사상 최대의 제국을 지배한 위대한 황제

건륭제 시기 중국이 세계 최대의 제국이었다는 중국 역사학계의 평가는 결코 허세가 아니다. 건륭황제는 재위 60년 동안 정치를 비롯해 경제와 문화 등 거의 모든 면에서 괄목할 만한 발전을 이루어 청나라를 확고한 반석 위에 올려놓았다. 21세기 중국이 ‘강건성세’康乾盛世의 부활을 꿈꾸는 것은 그런 이유에서다.

건륭황제는 자신의 부친인 옹정과도 닮지 않았고, 조부인 강희와도 다른 인물이다. 옹정이 비명에 죽고 그 보위를 승계할 때 건륭의 나이는 스물다섯 살이었다. 어린 나이에 제위에 오른 강희와는 달리 순조롭게 정권을 승계 받은 행운아였다. 비록 선대의 두 황제를 우상과 표본으로 받들고 따라가는 노력을 보이긴 했지만 강희와 옹정에 비해 건륭은 자기 나름대로의 통치철학을 구현했다.

건륭은 조부와 부친 세대를 두루 걸치며 어릴 적부터 어깨 너머로 권력의 암투를 보면서 통치술을 배웠다. 대권을 둘러싸고 벌어지는 집안싸움을 먼발치에서 지켜보며 지혜로운 대응책을 익혔고, 안安과 위危, 득得과 실失, 승勝과 패敗를 앞두고 현명하게 대처했던 선제들의 권모술수를 터득했다. 결코 평범하지 않은 그의 성장기는 그에게 큰 포부를 심어주었고, 꿈과 야망을 갖게 했다. 따라서 건륭은 청나라의 극성시대를 열어갈 큰 꿈을 안고 옹정 때의 폐정을 혁신할 정책을 실시하였다. 탐관오리들과의 전쟁을 치르면서 용단을 내려 탐묵貪墨에 연루된 측근대신들을 주살함으로써 이치쇄신에 대한 단호한 의지를 보이기도 했다.

성격적인 면에서도 건륭은 제왕의 자질이 넘쳤다. 문무를 겸비하고 지혜와 용맹을 구비했다. 옹정처럼 편집증을 보이지도 않았을 뿐더러 매사에 침착하고 멋과 풍류를 알았다. 이런 성격은 ‘위정이관’爲政以寬이라는 그의 통치술과도 일맥상통한다.

성세盛世의 화려함과 낙하落霞의 애절한 이야기

그러나 그의 긍정적인 성격들은 불행하게도 부패한 봉건제왕의 한계를 넘지 못하는 데 일조하고 말았다. 결국 순조롭게 출발했던 건륭의 시대는 망망대해에서 암초와 풍랑을 만나게 된다. 날로 더해만 가는 토지겸병과 탐관오리들의 비리와 부패는 독버섯처럼 퍼져 갔다. 그로 인해 국가는 속으로 병들어 훗날의 비극을 잉태했다. 결국 말년의 건륭은 화신和?과 같은 간악한 자들의 허와 실을 간파하지 못하고 현신賢臣들을 배척하면서 그의 제국은 급격히 쇠락의 길로 접어들게 되었다.

이렇듯 소설 《건륭황제》는 오늘을 사는 우리에게 한 나라가 흥망성쇠의 길을 걷는 과정을 적나라하게 묘사함으로써 멸망으로 치달을 수밖에 없었던 대제국의 역사를 극명하게 보여준다. 얼웨허는 장장 2천년의 세월을 지속해온 중국 봉건사회가 본격적으로 피폐해가는 마지막 백년의 모습을 서산으로 넘어가기 직전 붉게 물든 낙하落霞의 현란함에 비유했다. 그런 면에서 《건륭황제》는 어떻게든 살아남으려고 몸부림을 쳤지만 결국엔 거대한 변화의 물결에 밀려 역사의 피안으로 사라지고 만 중국 최전성기의 장엄한 대서사시이면서 눈물겨운 비가悲歌이기도 하다.

얼웨허의 소설을 읽으면 중국의 과거와 미래가 보인다

작가 얼웨허가 역사를 해부함에 있어서 특히 가상한 점은 여러 제왕들을 바라보는 시각이다. 소설 《건륭황제》는 오늘을 사는 우리에게 중국 최전성기의 화려함을 보여준다. 그 과정에서 그는 결코 자신의 사사로운 감정에 얽매여 제왕을 의도적으로 미화하거나 매도하지 않았다. 제왕들을 무조건 칭송하지도 않았을 뿐 아니라 예리한 칼날을 마구 휘둘러 기분 내키는 대로 잘라내지도 않았다. 처음부터 끝까지 냉정한 시각으로 역사를 직관하고 투시하는 자세로 일관했다. 천하의 군주라도 인간세상의 연화煙火를 먹고 사는 인간이라는 데 초점을 맞췄다. 때문에 작품 속에서 제왕들은 유아독존의 군주이기 전에 할머니를 보면 응석도 부리고 아들의 죽음 앞에선 눈물도 보일 줄 아는 인간으로 그려졌다. 또한 역사적인 사실에 입각하여 특별한 시대를 산 평범하지 않은 인물들을 무대 위로 끌어내 생생하게 살려냈다.

《건륭황제》는 총 6부로 구성되어 있다. 제목은 각각 풍화초로風華初露, 석조공산夕照空山, 일락장하日落長河, 천보간난天步艱難, 운암풍궐雲暗風闕, 추성자원秋聲紫苑이다. 제목만 봐도 처량하고 비극적인 기운이 갈수록 짙어지는 걸 알 수 있다. 이 또한 작가의 역사적인 시각과 깨달음이라고 할 수 있겠다.

-에디터가 말하는 『건륭황제』를 읽어야 하는 이유
‘물극필반’物極必反이라는 말이 있다. 모든 사물은 극성기에 이르면 뒤이어 반드시 쇠락하게 된다는 뜻이다. 건륭제 시대가 그랬다. 중국 역사상 가장 찬란한 부와 문화 발전을 이룬 ‘성세’盛世였기에 필연적으로 ‘낙하’落霞의 길로 접어들었다. 때문에 일부 역사학자들은 ‘강건성쇠’康乾盛衰로 부르기도 한다.

소설 《건륭황제》는 대청제국의 전성기를 지배한 건륭에 대한 방대한 기록이다. ‘성세盛世의 비가悲歌’라는 평가처럼 소설적 재미도 뛰어날 뿐 아니라 중국의 역사, 사회, 문화적 지식과 제왕의 통치철학까지 엿볼 수 있다. 그런 점에서 중국의 과거와 미래를 알기 위해서는 반드시 읽어보아야 할 역사소설이다. 백년 후에도 미국이 우리의 우방일지는 장담할 수 없지만 중국은 여전히 이웃으로 남아 있을 것이기 때문이다.

-번역자 홍순도의 ‘옮긴이의 말’ 중에서

중국에서 ‘역사소설의 황제’로 불리는 얼웨허二月河의 ‘제왕삼부곡’帝王三部曲 시리즈 중 마지막인 《건륭황제》의 주인공 건륭은 ‘강건성세’康乾盛世라는 말에서 보듯 그는 세가 아니다. 21세기 중국이 건륭시대의 영광을 꿈꾸는 것도 바로 그런 이유에서다.

그러나 공功만큼 과過도 적지 않았다. 특히 말년에 이르러서는 자신에 대한 신격화가 점점 심해졌다. 이는 주변 소수민족과 국가를 대상으로 열 번 싸워 모두 이겼다는 자부심을 담은 별칭 ‘십전노인’十全老人에 대한 애착이 지나칠 정도였다는 사실에서도 잘 알 수 있다. 그러다보니 갈수록 독선과 아집에 빠지게 되었다. 또 제국 밖에도 세상이 있다는 사실을 인정하지 않은 채 폐관쇄국閉關鎖國 정책을 실시해 청나라가 더욱 발전할 수 있는 기회를 스스로 걷어찼다. 이후 세계 최강대국 중국은 서서히 몰락의 길로 접어들기 시작했다.

총 6부로 구성된 《건륭황제》는 대청제국의 전성기와 몰락의 시대에 대한 방대한 기록으로서 전작인 《강희대제》와 《옹정황제》를 합친 것에 버금갈 정도의 분량이다. 작가 얼웨허가 1992년부터 시작해 무려 8년이라는 장구한 세월 동안 집필을 이어간 것은 기나긴 건륭 연간에 어마어마하게 많은 인물과 사건들이 어우러진 복잡다단한 이야기가 있었기 때문이다. 그러기에 이 작품은 대작이 많은 중국에서도 흔히 볼 수 없는 대작 중의 대작이다. 하지만 소설로서의 재미도 크거니와 21세기 세계의 중심으로 부상하고 있는 중국의 사회, 문화, 역사에 대한 배경지식과 제왕의 통치철학에 대해서도 배울 수 있다는 점에서 통독의 가치는 크다고 생각한다.""")

역사 거울 천하 흥망성쇠 미래 중국 최고 지도부 선택 치세 지침서 제왕 삼부 곡 완결 중국 꿈 세계 최 강국 건륭 시대 부활 작가 얼웨허는 제왕 삼부 곡 시리즈 발표 시작 단원 마지막 강희 옹정 건륭 황제 얼웨허는 한자 로 제왕 삼부 곡 시리즈 완성 중국 문학사 발자취 문학 측면 때 얼웨허의 제왕 삼부 곡 성숙 세련미 자랑 건륭 황제 향기 발산 소설 속 인물 말 강희 창세 지조 이고 옹정 입국 지조 건륭 개업 지주 건륭 황제 얼웨허의 제왕 삼부 곡 작가 심혈 역작 인 역사 최대 제국 지배 위대 황제 건륭제 시기 중국 세계 최대 제국 중국 역사학 평가 허세 건륭 황제 재위 동안 정치 비롯 경제 문화 면 괄목 발전 청나라 확고 반석 위 중국 강건성세 의 부활 이유 건륭 황제 자신 부친 인 옹정 조부 인 강희 와도 인물 옹정 비명 보위 승계 때 건륭 나이 제위 강희 정 승계 행운아 선대 황제 우상 표본 노력 강희 옹정 건륭 대로 통치 철학 구현 건륭 조부 부친 세대 어깨 너머 권력 암투 통치 술 대권 집안싸움 먼발치 지혜 대응책 과 위 득 과 실 승 과 패 현명 대처 선제 권모술수 터득 평범 성장기 포부 꿈 야망 건륭 청나라 극성 시대 꿈 안고 옹정 때 정 혁신 정책 실시 탐관오리 전쟁 용단 탐 묵 연루 측근 대신 주 살 이치 쇄신 단호 의지 성격 면 건륭 제왕 자질 문무 겸비 지혜 용맹 구비 옹정 편집증 매사 침착 멋 풍류 성격 위정 이관 통치 술 일맥상통 성세 의 화려 낙하 의 애절 이야기 긍정 성격 불행 부패 봉건 제왕 한계 일조 출발 건륭 시대 망망대해 암초 풍랑 토지 겸병 탐관오리 비리 부패 독버섯 인 국가 속 훗날 비극 잉태 말년 건륭 화신 과 간악 허 실 간파 현신 배척 제국 쇠락 길 소설 건륭 황제 오늘 나라 흥망성쇠 길 과정 적나라 묘사 멸망 제국 역사 극명 얼웨허는 세월 지속 중국 봉건 사회 본격 피폐 마지막 모습 서산 직전 낙하 의 현란 비유 면 건륭 황제 몸부림 결국 거대 변화 물결 역사 피안 중국 최 전성기 장엄 서사시 비 이기 얼웨허의 소설

In [9]:
make_keywords("""“세상은 내 기대에 미치지 못했어. 떠나보면 달라질까?”
『고슴도치의 소원』 톤 텔레헨의 여행 같은 소설


“사람들은 어떻게 떠날 생각을 잊은 채 살아가지?”
떠나기로 결심하고, 계속 망설이고, 다시 먼 곳을 꿈꾸는 그 모든 마음에 귀를 기울이는 여행

어느 날 코끼리가 말했다.
“나 사막으로 떠나려고 해. 언제 돌아올지는 나도 잘 모르겠어.”
“갑자기 왜?” 다람쥐가 놀라 물었다.
“거기에 가보면 이유를 찾게 될지도 모르지.”
다람쥐는 달콤한 너도밤나무 껍질을 배낭에 싸서 코끼리 등에 메어 주었다.

“잘 다녀와, 코끼리야.”



◎ 도서 소개

“세상은 내 기대에 미치지 못했어. 떠나보면 달라질까?”
『고슴도치의 소원』 톤 텔레헨의 여행 같은 소설

동물들의 이야기를 통해 현대인들의 복잡한 내면을 들여다보는 톤 텔레헨의 소설 『잘 지내니』와 『잘 다녀와』가 아르테에서 출간되었다. 현대인의 고독을 고슴도치에 빗대어 표현한 소설 『고슴도치의 소원』, 하늘을 날겠다는 새로운 도전을 하지만 매번 나무에서 떨어지고 마는 코끼리 이야기 『코끼리의 마음』에 이은 어른을 위한 소설 시리즈다. 앞선 작품들과 마찬가지로 원서에는 없는 RASO(김소라)의 일러스트가 포함되어 사랑스러운 그림을 보는 재미가 더해졌다.
『잘 다녀와』속 동물들은 언젠가 숲속 일상을 떠나볼 생각을 품고 있다. 왠지 먼 곳엔 특별한 게 있을 것만 같다. 가본 적이 없는 미지의 세계이기 때문이다. 그러나 숲 밖 여정은 만만치 않다. 사막과 바다, 그리고 파라다이스조차. “솔직히 말하면, 그냥 집에 있는 게 편할 수 있지. 그 힘든 여정들을 생각하면…….”
코끼리는 떠나는 이유를 찾기 위한 여행을 떠나고, 다람쥐는 배낭을 다 싸고서도 여행을 갈지 말지 계속해서 망설인다. 개미와 다람쥐가 끝내 떠난 여행에서 크나큰 벽을 맞이하고서 절망하고, 개미는 세상은 내 기대에 미치지 못했다며 투덜거린다. 개구리는 먼 곳에 가도 별 게 없다는 걸 깨닫지만, 먼 곳에 가봤다는 사실만으로 기쁨을 느낀다.


"사람들은 어떻게 떠날 생각을 잊은 채 살아가지?”
떠나기로 결심하고, 계속 망설이고,
다시 먼 곳을 꿈꾸는 그 모든 마음에 귀를 기울이는 여행

우리는 늘 이곳이 아닌 저곳을 꿈꾼다. 일상에 지칠 때면 어딘가 새로운 곳으로의 여행을 계획한다. 하지만 여행을 떠나는 일도쉬운 일만은 아니다. 어쩌면 아주 큰 용기가 필요한 일일지도 모른다. 어디론가 떠나고 싶기도 하고 지금 여기에 머무르고싶은 마음이 공존한다. 용기 내어 떠난다고 해도, 어느 순간엔 편안하고 익숙한 곳으로 돌아가고 싶은 마음이 든다.
떠나면 돌아오고 싶고, 돌아오면 또 떠나고 싶어지곤 한다. 톤 텔레헨은 동물들의 작은 이야기들을 통해 이런 마음까지도 모두 여행의 과정이라고 말한다. 익숙한 곳에서 새로운 것을 꿈꾸며 설레하는 마음, 낯선 환경에서 편안하고 익숙한 집을 떠올리는 모든 마음까지도. 이런 모든 여행의 과정이 우리의 삶과 닮아 있다.
꼭 여행이 아니더라도 우리는 인생을 살아가면서 항상 새로운 것을 찾는다. 매일 똑같은 일상은 지루해지기 마련이니까. 반면에 익숙하고 편안한 것이 주는 안정감 또한 버릴 수 없다. 결정을 내리지 못하고 늘 망설이기도 한다. 하지만 텔레헨의 이야기는 도전하거나 안주하거나, 떠나거나 돌아오는 모든 일들이 모두 의미 있다고 위로한다. 이상을 꿈꾸며 먼 곳에 갔어도 내가 원하는 것을 발견하지 못할 수 있다. 그래도 괜찮다, 먼 곳에 가봤으니까. 그곳에 가봤다는 사실 자체로 이전의 나와는 달라졌을 테니까.

밤이 되자 개구리가 집으로 돌아왔다. 먼 곳은 실망스러웠다. 아주 가까이, 정말 코앞에 가서 보았다. 그러나 뭔가 특별한 걸본 건 아니었다. 사실 뭐 아무것도 없었다. 그러나 그 먼 곳에 가 봤다는 것만으로도 개구리는 기뻤다.(본문 43쪽)

『잘 다녀와』는 여행을 꿈꾸고, 망설이고, 떠나는 이들에게 선물 같은 책이 될 것이다.""")

세상 기대 고슴도치 소원 텔 레 의 여행 소설 사람 생각 결심 곳 마음 귀 여행 날 코끼리 말 사막 다람쥐 놀라 이유 다람쥐 달콤 너도밤나무 껍질 배낭 코끼리 코끼리 도서 소개 세상 기대 고슴도치 소원 텔 레 의 여행 소설 동물 이야기 현대 인 복잡 내면 텔 레 의 소설 아르 테 출간 현대인 고독 고슴도치 표현 소설 고슴도치 소원 하늘 도전 나무 코끼리 이야기 코끼리 마음 어른 소설 시리즈 작품 마찬가지 원서 김 소라 일러스트 포함 사랑 그림 재미 속 동물 숲 속 일상 생각 곳 특별 게 미지 세계 숲 밖 여 정 만만 사막 바다 파라다이스 말 집 게 여 정 생각 코끼리 이유 여행 다람쥐 배낭 여행 계속 개미 다람쥐 여행 벽 맞이 절망 개미 세상 기대 개구리 곳 가도 게 곳 사실 기쁨 사람 생각 결심 곳 마음 귀 여행 일상 때 곳 여행 계획 여행 용기 필요 마음 공존 용기 순간 편안 익숙 곳 마음 텔 레 은 동물 이야기 마음 여행 과정 말 익숙 곳 마음 낯선 환경 편안 익숙 집 마음 여행 과정 삶 여행 인 일상 지루 마련 반면 익숙 편안 안정감 결정 텔 레 의 이야기 도전 안주 일 의미 위로 이상 곳 발견 곳 사실 자체 이전 테 밤 개구리 집 곳 실망 코앞 특별 본 사실 아무것 곳 개구리 본문 여행 선물 책


In [10]:
make_keywords("""원시의 아프리카 문학을 읽는 시간!
급격히 밀려들어 온 미국 문화의 물줄기, 세계화라는 이름이 붙은 그 거센 흐름을 마주한 사람들
동경과 환멸, 몰이해와 소통의 순간들을 오가며 공존에 다다르려는 주변인들의 위태하고도 흥미로운 여정

아프리카 현대 문학을 이끄는 차세대 대표 작가 치마만다 응고지 아디치에의 신작 소설이다. 아디치에는 『자주색 히비스커스』로 등단하자마자 “치누아 아체베의 21세기 딸”이라는 명성을 얻었으며, 두 번째 장편소설 『태양은 노랗게 타오른다』(2006)로 오렌지 소설상을 받고 “천재 상”이라 불리는 맥아서 펠로로 선정되었다. 『숨통』은 2002년부터 6년간 《프로스펙트》, 《그란타》등 세계 유수의 잡지에 발표했던 열두 개의 단편들을 모은 소설집으로, 모든 것이 세계화라는 명목으로 ‘미국화’되어 가는 세상에서 전통을 지키려 애쓰며 자신만의 삶의 양식을 개척해 가는 나이지리아인들의 지난한 여정을 매우 사실적으로 그려 냈다.

전작들을 통해 고국 나이지리아가 겪은 역사의 진실을 전 세계에 알리는 데 힘썼던 작가는 이번에는 열아홉 살에 도미한 자전적 경험을 바탕으로 미국인이 아닌 ‘타국인’이라면 누구나 공감할 만한, 보다 동시대적이고 보편적인 이야기를 들려준다. 세계화라는 커다란 흐름에 놓인 약소국가의 한 개인이 주체성을 잃지 않기 위해 분투하는 과정을 담아낸 『숨통』의 이야기들은 우리에게도 시사해 주는 바가 크다. 작가의 날카롭고 섬세한 관찰력과 진중하면서도 곳곳에 배어 있는 유머가 돋보이는 이 작품은 2009년 《파이낸셜 타임스》 선정 ‘올해의 도서’ 목록에 올랐다. 아디치에는 2011년 《뉴요커》에서 뽑은 ‘미국을 대표하는 젊은 소설가 20인’과 하버드 대학교 래드클리프 고등 연구소 펠로로 선정되기도 했다. 그런 질문을 하는 동안 그의 눈은 눈물 어린 꿈으로 빛났다. “우리도 언젠가는 아드모어나 메인 라인의 다른 동네에 있는 그런 집에서 살게 될 거야.” 그가 말했다. 그녀는 아무 말도 하지 않았다. 그녀에게 중요한 것은 그들이 어디에 사느냐가 아니라 변해 버린 그들의 모습이었기 때문이다. ---「지난주 월요일에」중에서

당신에게 아프리카 어느 나라에서 왔냐고 물었을 때, 당신은 나이지리아라고 대답하고 나서 그가 보츠와나의 에이즈 퇴치 운동에 돈을 기부한 얘기를 하길 기대했다. 하지만 그는 당신이 요루바족인지 이보족인지를 물었다. 얼굴을 보니 풀라니족은 아니라는 것이다. 당신은 깜짝 놀랐다. (중략) 그는 가나와 우간다와 탄자니아에 가 본 적이 있고, 오코트 프비테크의 시와 아모스 투투올라의 소설을 좋아하며, 사하라사막 이남의 나라들과 역사와 사회문제에 대한 책을 많이 읽었다고 했다. 당신은 경멸감을 느끼고 싶었고, 그가 주문한 음식을 가져다주면서 그 경멸감을 보여 주고 싶었다. 왜냐하면 아프리카를 너무 좋아하는 백인들과 너무 싫어하는 백인들은 똑같은 부류, 즉 찰난 척하는 부류였기 때문이다. ---「숨통」중에서

“부인? 미합중국은 정치적 박해의 피해자에게는 새로운 삶을 제공합니다만 그러려면 반드시 증거가…….” 새로운 삶. 그녀에게 새로운 삶을 줬던 것은 우곤나였다. 그녀는 자신이 새로운 신분, 새로운 정체성에 적응해 가는 속도에 깜짝 놀랐었다. “제가 우곤나 엄마예요.” 그녀는 유치원에서, 교사들에게, 다른 학부모들에게 그렇게 말하곤 했다. (중략) 꽃이 피어서 벌이 모여들면 그녀는 땅 위에 도두앉은 채 꽃을 따서 빨아 먹고 싶었다. 그리고 나중에 다 먹은 꽃들을, 우곤나가 레고 블록으로 그랬듯이, 일렬로 나란히 늘어놓고 싶었다. 그녀는 깨달았다. 그것이 그녀가 원하는 새로운 삶이라는 것을. (중략) 그녀는 돌아서지 않았다. 그녀는 미국 대사관을 나와서, 아직도 법랑 접시를 한껏 앞으로 내민 채 구걸을 하고 있는 거지들을 지나 자기 차에 올라탔다. ---「미국 대사관」중에서

그날은 신부가 미사를 시작하면서 성수로 신도들을 축복하는 일요일이었기 때문에 패트릭 신부가 왔다 갔다 하면서 큰 소금 통처럼 생긴 것으로 사람들에게 물을 뿌렸다. 우카마카는 그를 쳐다보면서, 미국의 가톨릭 미사가 나이지리아보다 얼마나 억제되어 있나 생각했다. 나이지리아에서였다면, 땀 흘리며 허둥지둥하는 복사(服事)가 든 성수 통에 신부가 담근 것은 망고 나무에서 꺾은 싱싱한 녹색 가지였을 것이고, 신부는 성큼성큼 걸어 다니면서 물을 뿌리고 빙글빙글 돌아서 성수가 비 내리듯 했을 것이며, 사람들은 흠뻑 젖은 채 미소를 띠고 성호를 그으면서 자신들이 정말로 축복받았다고 느꼈을 것이다. ---「전율」중에서

그가 카트에 포장육을 담았을 때 나는 신경이 바짝 곤두섰다. 나는 오그베테 시장에서 자주 하던 것처럼 고기를 직접 만져 보고 색깔이 얼마나 빨간지 자세히 들여다보고 싶었다. 그곳에서는 푸주한이 방금 자른 고기를 들어 보이면 파리가 그 주위를 윙윙대곤 했다. “저 비스킷 좀 사도 돼요?” 내가 물었다. 버턴스 리치 티의 파란 포장이 눈에 익어서였다. 비스킷을 먹고 싶은 건 아니었지만 카트에 뭔가 익숙한 게 있었으면 했다. “쿠키. 미국인들은 그걸 쿠키라고 불러요.” 그가 말했다. 나는 손을 뻗어서 비스킷(쿠키)을 집었다. (중략) “영어로 말해요. 당신 뒤에 사람들 있어요.” 그가 반짝이는 보석으로 가득한 유리 진열장 쪽으로 나를 끌어당기며 속삭였다. “그리고 그건 엘리베이터예요, 리프트가 아니라. 미국인들은 엘리베이터라고 해요.” ---「중매인」중에서

느왐그바가 그 애를 안은 순간부터 아기의 밝은 눈동자는 생글거리며 그녀를 쳐다봤고 그녀는 오비에리카의 영혼이 돌아왔음을 알았다. 여자애로 환생하다니 이상한 일이었지만 조상님의 뜻을 누가 예측할 수 있겠는가? 오도널 신부는 아기에게 그레이스라는 세례명을 주었지만 느왐그바는 손녀를 ‘내 이름이 영원히 사라지지 않게 하소서.’라는 뜻의 아파메푸나라고 불렀다. 꼬마 손녀가 그녀의 시와 이야기에 진지한 관심을 보이고, 소녀가 되어서도 할머니가 떨리는 손으로 힘들게 도자기를 만드는 모습을 주의 깊게 지켜보는 데 느왐그바는 전율을 느꼈다. 하지만 그녀는 아파메푸나가 중등학교에 가게 되었을 때는 전율을 느끼지 않았다. 왜냐하면 기숙학교에서 배우게 될 새로운 방식들이 손녀의 투지를 사라지게 하고 아니켄와 같은 옹고집이나 음그베케 같은 무력함으로 바꿔 놓을까 봐 겁났기 때문이다.
---「고집 센 역사가」 중에서""")

원시 아프리카 문학 시간 미국 문화 물줄기 세계 이름 흐름 사람 동경 환멸 몰이해 소통 순간 공존 주변인 위태 흥미 여 정 아프리카 현대 문학 차세대 대표 작가 치마 응고 아 디 신작 소설 아 디 자주색 히비스커스로 등단 치누아아체베 딸 명성 장편소설 태양 오렌지 소설 천재 상 맥 아서 펠로로 선정 숨통 프로 스펙 세계 유수 잡지 발표 단편 소설집 세계 명목 미국화 세상 전통 자신 삶 양식 개척 나이지리아 인 지난 여 정 사실 전작 고국 나이지리아 역사 진실 세계 작가 이번 도미 자전 경험 바탕 미국 인 타국 인 공감 한 동시대 보편 이야기 세계 흐름 약소국가 개인 주체성 분투 과정 숨통 이야기 시사 바가 작가 섬세 관찰력 진중 하면 서도 곳곳 유머 작품 파이낸셜타임스 선정 올해 도서 목록 아 디 뉴요커 미국 대표 소설가 하버드대학교 래 드 리프 고등 연구소 펠로로 선정 질문 동안 눈 눈물 꿈 아 드 모어 메인 라인 동네 집 말 아무말도하지않았다 중요 모습 지난주 월요일 중 아프리카 나라 때 나이지리아 라고 대답 보츠와나 에이즈 퇴치 운동 돈 기부 얘기 기대 요루바족 인지 이보족 인지 얼굴 보니 족 중략 가나 우간다 탄자니아 코트 프비테크의 시와 아모스 투투 소설 사하라사막 남 나라 역사 사회 문제 책 경멸감 주문 음식 경멸감 아프리카 부류 찰나 부류 숨통 중 부인 미 합중국 정치 박해 피해자 삶 제공 증거 삶 삶 우 곤 자신 신분 정체성 적응 속도 제가 우 곤 엄마 유치원 교사 학부모 말 중략 꽃 벌이 땅 위 도 꽃 나중 꽃 우 곤 나가 레고 블록 일렬 삶 중략 미국 대사관 법랑 접시 앞 내민 구걸 거지 차 미국 대사관 중 그날 신부 미사 시작 성수 신도 축복 일요일 패트릭 신부 소금 통 사람 물 우 카 마카 미국 가톨릭 미사 나이지리아 억제 생각 나이지리아 땀 복사 성수 통 신부 망고 나무 싱싱 녹색 신부 물 빙글빙글 성수 비 사람 미소 성호 자신 축복 전율 중 카트 포 장육 때 신경 베 테 시장 고기 색깔 이 고기 파리 주위 비스킷 사도 버턴스 리치 티 

In [11]:
make_keywords("""아시아 클래식' 5권. '페르시아어의 아버지'라 불리는 아볼 카셈 피르다우시가 35여 년에 걸쳐 완성한 페르시아 문학의 영원한 고전이자 베스트셀러 <샤나메>. '왕의 책' 또는 '왕들의 책'이라는 뜻의 이 책은 창세부터 7세기 이슬람의 침입으로 멸망하기 전까지, 이란의 신화.전통.역사가 담겼다. 페르시아 언어 세계에서 베르길리우스의 <아이네이스>, 호메로스의 <일리아드>, 단테의 <신곡>, 존 밀턴의 <실락원>과 비견되곤 한다.

아랍의 지배자들이 아랍어와 아랍문화를 강요하는 상황에서도, 피르다우시는 페르시아어로 페르시아의 전설.신화와 역사를 기록했고 이는 페르시아인들의 영원한 자랑이다. 그래서 페르시아인들은 이 작품이 페르시아인들의 심장에 자리 잡았다고 말하며, 피르다우시를 인류 역사상 최고의 대문호로 일컬어지는 괴테와 셰익스피어, 호메로스에 견줄 수 있다고 하는 것이다.

세계에서 가장 오래된 신화로 알려진 수메르의 <길가메시 서사시>, 인도 정신문화를 지탱하는 두 기둥인 <라마야나>와 <마하바라타>, 유럽인의 정신과 사상의 원류인 고대 그리스의 서사시 <일리아드>와 <오디세이아> 등에서 보이는 재미와 상상력의 보고를 <샤나메>에서도 엿볼 수 있다. 한국 독자만 읽지 않은 세계적 고전 『샤나메』 국내 최초 출간!
이란 건국 신화와 역사가 담긴 ‘왕의 책’

『샤나메』의 영어 번역은 19세기 초부터 시도되었는데, 대부분 축약본이다. 제임스 앳킨슨이 1832년에 첫 번째 영어 역본을 출간했고, 헬렌 짐머른이 1883년 영어 역본을 출간했으며, 1905년에는 아서 조지 워너와 에드먼드 워너 형제가 영어 역본을 출간한 바 있다. 서구에서는 일찍이 『샤나메』가 고전의 반열에 올랐음을 알 수 있다.

본서는 1883년 헬렌 짐머른의 영어 역본을 저본으로 삼았다. 니체의 친구이기도 한 헬렌 짐머른은 작가이자 번역가로 활동하며 예술, 역사, 철학, 소설, 드라마 등 장르를 불문하고 다양한 방면에서 재능을 꽃피웠다. 그런 그가 『샤나메』를 선택했다는 건 가치다운 가치를 발견했기 때문이겠다.

한편 중국에서는 『열왕기』, 일본에서는 『왕서』라는 제목으로 번역본이 존재한다. 우리나라에는 『샤나메』라는 제목을 단 번역본은 없다. 다만 몇몇 책에서 『샤나메』의 일부 대목들을 확인할 수 있을 뿐이다. 본서가 우리나라에서의 최초 번역서가 되겠다.

국내에는 그동안 왜 『샤나메』가 번역 출간이 되지 않았었는가? 서구에서는 다양한 장르로 재창조된 이 고전이 한국에서는 그 제목조차 낯선 이유는 서구가 아시아의 풍부한 이야기 콘텐츠에 눈을 돌려 실속을 챙기는 사이, 우리는 여전히 서구에서 눈을 떼지 못하고 있기 때문일 것이다. 서구만 쫓다보니 정작 우리네 아시아의 콘텐츠에는 눈을 돌리지 않았다. 이제라도 눈을 돌려 일방적인 상상력의 메마름에 활력을 활기를 불어넣고, 정체성을 찾아야 할 때다. 그 시작을 함께할 콘텐츠로 『샤나메』는 최적이다.

페르시아 문화의 백과사전, 페르시아의 정전(正典)!
4왕조 50여 명의 왕과 영웅 들이 펼치는 탐욕과 파멸, 생명의 서사시

2500여 년의 이란 역사 중, 『샤나메』는 창세부터 7세기 이슬람의 침입으로 멸망하기까지 약 1200년을 다루고 있다. 장구한 세월에 걸쳐 4왕조 50여 명의 왕과 영웅 들이 탐욕과 파멸, 생명의 서사시를 펼친다.

무수히 등장하는 영웅들 중 단연 압권은 ‘루스템’이다. 샤(왕)로부터 영토를 하사받고, 전쟁 시 군대를 징집해 지휘하여 샤를 위해 싸우는 ‘펠리바’인 루스템은 샤와 나라가 위험에 처할 때마다 단숨에 달려가 여지없이 적들을 해치우곤 한다. 그가 경험하는 모험, 사랑, 고통, 슬픔, 생명, 죽음의 일련을 따라가다 보면 인간사의 단면을 목도하게 된다. 또한 그와 함께 하는 또는 적대하는 왕과 영웅 들의 이야기는 수많은 이야기꾼의 입에 오를 만하다.

『샤나메』에서 가장 슬픈 장면을 뽑으라면, 단연 루스템과 그의 아들 소랍의 대결이다. 서로의 존재를 알지 못하고 살아온 부자(父子)가 운명의 장난으로 전쟁터에서 만나 서로 싸우게 되고 결국 루스템이 아들 소랍을 제 손으로 죽이게 되는 이 이야기는, 19세기 영국의 시인이자 평론가인 매슈 아널드가 시로 쓰기도 하였다. 할레드 호세이니의 세계적인 베스트셀러 『연을 쫓는 아이』에서 주인공 아미르가 읽어주었던 「루스템과 소랍」 이야기는 『샤나메』에서 루스템과 소랍의 비극적 대결을 말하는 것이다.

이런 모험담과 함께 몇몇 이야기는 이란을 넘어 타지키스탄, 터키, 아프가니스탄 등 다른 지역에도 널리 전파되었다. 아울러 연극, 영화, 춤, 만화, 그림, 음악 등 여러 장르에 소재를 제공했으며, 영화로 제작되기도 한 고전 게임 <페르시아의 왕자>를 비롯해 여러 컴퓨터 게임 등의 IT 산업과 결합하여 늘 새로운 모습으로 우리들 앞에 보이곤 한다.

한편 『샤나메』는 봉변을 당한 적도 있다. 1979년 혁명으로 팔레비 왕조를 축출하며 이란공화국을 건국한 이슬람 근본주의 정부에 의해 마샤드의 벽화로 묘사된 서사시 『샤나메』가 지워졌던 것이다. 하지만 현재는 유아, 초등학생, 중고등학생들이 교과서로 활용되고 있을 뿐 아니라 페르시아 문화를 이해할 수 있는 중요한 자료로 주목받고 있다.

‘페르시아어의 아버지’ 아볼 카셈 피르다우시
페르시아 인들이 가장 사랑하고 존경하는 작가와 작품!

『샤나메』는 이란의 건국 신화와 역사 그리고 고대 신앙으로서의 조로아스터교를 담은 책으로 페르시아 문화의 백과사전, 페르시아의 정전(正典)이라 할 만하다. 구체적으로 피슈다디 왕조, 카야니 왕조, 아슈카니 왕조, 사산 왕조의 네 왕조의 역사를 기록하고 있다. 앞의 두 왕조는 신화와 전설에 바탕을 두었고, 뒤의 두 왕조는 역사적 사실을 바탕으로 기록하고 있다. 영웅, 사랑, 전쟁, 모험, 환상 등 다양하고 흥미진진한 이야기들이 쉴 새 없이 전개되는바, 내용은 다이내믹해 지루함을 모른다.

이런 역사적, 문학적 중요성 외에도 『샤나메』는 ‘페르시아어의 아버지’라 불리는 아볼 카셈 피르다우시에 의해 순수한 페르시아어로 쓰여 페르시아어를 되살리는 구심점이라는 점도 있다. 피르다우시가 이 작품을 쓸 당시는 아랍에 의해 페르시아가 지배당하고 있었기 때문에, 아랍어와 아랍문화의 강요가 계속되고 있었다. 페르시아 문화와 역사, 언어가 점차 사라지고 있었던 것이다. 페르시아 인들이 피르다우시를 생각하는 마음이 절절히 전해진다.

이처럼 『샤나메』는 페르시아 인들이 가장 사랑하는 작가의 작품임과 동시에 그 자체로도 가장 사랑받는 작품이다. 일례로 페르시아 세밀화로 화려하게 장식한 『샤나메』 삽화본들 중 어떤 그림은 한 장에 90만 파운드, 우리 돈으로 약 15억 원에 팔리기도 할 만큼 예술적 가치를 인정받고 있다.

화려했던 중세 페르시아 문화의 결정체라 할 이 책은 지금은 존재하지 않는 중세 페르시아어로 집필되어 언어학적인 측면에서도 그 가치를 높이 평가받는다. 그를 ‘페르시아어의 아버지’라 일컫는 것도 이 때문이다. 사후 그의 유해는 그가 살던 투스 집의 정원에 묻혔는데, 복원되어 현재는 국가 기념물로 관리되고 있다.""")

아시아 클래식 페르시아어 아버지 카 피르다우시 완성 페르시아문학 영원 고전 이자 베스트셀러 샤나메 왕 책 왕 책 뜻 책 창세 이슬람 침입 멸망 전 이란 신화 전통 역사 페르시아 언어 세계 베르길리우스 아이네이스 호메로스 일리아 드 단테 신곡 존밀턴 실락원 비견 아랍 지배자 아랍어 아랍 문화 강요 상황 피르다우시 페르시아어 페르시아 전설 신화 역사 기록 페르시아인들 영원 자랑 페르시아인들 작품 페르시아인들 심장 자리 말 피르다우시 인 역사 최고 대문호 괴테 셰익스피어 호메로스 세계 신화로 수메르 길가메시서사시 인 정신문화 지탱 기둥 인 라마야나 마하바라타 유럽인 정신 사상 원류 인 고대그리스 서사시 일리아 드 오디세이아 재미 상상력 보고 샤나메 한국 독자 세계 고전 샤나메 국내 최초 출간 이란 건국신화 역사 왕 책 샤나메 영어 번역 시도 대부분 축약 본 제임스 앳킨슨 영어 역 본 출간 헬렌 짐머 르 영어 역 본 출간 아서 조지 워너 에드먼드 워너 형제 영어 역 본 출간 서구 샤나메 고전 반열 본서 헬렌 짐머 르 영어 역 본 저본 니체 친구 헬렌 짐머 르 작가 번역가 활동 예술 역사 철학 소설 드라마 장르 불문 다양 방면 재능 샤나메 선택 가치 다운 가치 발견 한편 중국 열왕기 일본 왕서 제목 번역본 존재 우리나라 샤나메 제목 번역본 책 샤나메 일부 대목 확인 본서 우리나라 최초 번역서 국내 그동안 샤나메 번역 출간 서구 다양 장르 창조 고전 한국 제목 낯선 이유 서구 아시아 풍부 이야기 콘텐츠 눈 실속 사이 서구 눈 서구 정작 아시아 콘텐츠 눈 돌리지 제라 눈 일방 상상력 활력 활기 정체성 시작 콘텐츠 샤나메 최적 페르시아 문화 백과사전 페르시아 정 전 왕조 왕 영웅 탐욕 파멸 생명 서사시 이란 역사 샤나메 창세 이슬람 침입 멸망 장구 세월 왕조 왕 영웅 탐욕 파멸 생명 서사시 등장 영웅들 압권 루 스템 샤 왕 영토 하사 전쟁 군대 징집 지휘 샤를 펠리바인 루 스템 샤와 나라 위험 때 경험 모험 사랑 고통 슬픔 생명 죽음 일련 인간사 단면 목도 적대 왕 영웅 이

In [12]:
make_keywords("""그들은 다른 군중들과 같이 양철 지붕의 큰 창고로 들어갔다. 비는 억수같이 퍼붓기 시작했다. 그들은 얼마간 그곳에 서 있었다. 그러자 ‘데디빋슨’씨가 왔다. 그는 여행을 하는 동안 ‘맥페일’ 부처에게 아주 공손하게 대했다. 그러나 그는 시간의 대부분을 독서로 보냈다. 그는 말이 없고 좀 우울한 편이었다. 그리고 그의 상냥함은 그가 기독교도의 의무로서 받아들인 것처럼 보였다. 그는 본래 말이 적고 무뚝뚝하기 조차했다. 그의 외모는 특이했다. 그는 키가 대단히 컸고 바싹 말랐으며 긴 사지는 헐렁하게 붙어 있었다. 그는 볼이 폭 패였고, 이상하게 광대뼈가 튀어나와 있었다. 그는 너무나도 창백한 모습을 하고 있었으므로 그의 입술만이 도톰하고 관능적(官能的)인 것을 볼 때 놀라지 않을 수 없었다. 그는 머리를 대단히 길게 기르고 있었다. 그의 크고 검은 눈은 비극적인 빛을 띠고 있었다. 그리고 그의 손은 손가락이 크고 길어서 보기에 좋았다. 그 손은 그가 비상한 힘을 갖고 있다는 인상을 주었다. 그러나 그에게서 가장 유별난 것은 보는 사람으로 하여금 그가 억눌린 정열을 갖고 있다고 느끼게 하는 것이었다. 그것은 인상적이었고 어딘지 모르게 불안감을 주었다. 그는 누구나 사귈 수 있는 사람이 아니었다. 그들은 다른 군중들과 같이 양철 지붕의 큰 창고로 들어갔다. 비는 억수같이 퍼붓기 시작했다. 그들은 얼마간 그곳에 서 있었다. 그러자 ‘데디빋슨’씨가 왔다. 그는 여행을 하는 동안 ‘맥페일’ 부처에게 아주 공손하게 대했다. 그러나 그는 시간의 대부분을 독서로 보냈다. 그는 말이 없고 좀 우울한 편이었다. 그리고 그의 상냥함은 그가 기독교도의 의무로서 받아들인 것처럼 보였다. 그는 본래 말이 적고 무뚝뚝하기 조차했다. 그의 외모는 특이했다. 그는 키가 대단히 컸고 바싹 말랐으며 긴 사지는 헐렁하게 붙어 있었다. 그는 볼이 폭 패였고, 이상하게 광대뼈가 튀어나와 있었다. 그는 너무나도 창백한 모습을 하고 있었으므로 그의 입술만이 도톰하고 관능적(官能的)인 것을 볼 때 놀라지 않을 수 없었다. 그는 머리를 대단히 길게 기르고 있었다. 그의 크고 검은 눈은 비극적인 빛을 띠고 있었다. 그리고 그의 손은 손가락이 크고 길어서 보기에 좋았다. 그 손은 그가 비상한 힘을 갖고 있다는 인상을 주었다. 그러나 그에게서 가장 유별난 것은 보는 사람으로 하여금 그가 억눌린 정열을 갖고 있다고 느끼게 하는 것이었다. 그것은 인상적이었고 어딘지 모르게 불안감을 주었다. 그는 누구나 사귈 수 있는 사람이 아니었다.""")

군중 양철 지붕 창고 시작 얼마간 데디빋슨씨가 여행 동안 맥 페이 부처 공손 시간 대부분 독서 말 우울 상냥 기독교도 의무 본래 말 무뚝뚝 조차 외모 특이 키 사지 헐렁 볼 폭 패 이상 광대뼈 창백 모습 입술 도톰 관능 때 머리 눈 비극 빛 손은 손가락 손은 비상 힘 인 사람 정열 인상 불안감 사람 군중 양철 지붕 창고 시작 얼마간 데디빋슨씨가 여행 동안 맥 페이 부처 공손 시간 대부분 독서 말 우울 상냥 기독교도 의무 본래 말 무뚝뚝 조차 외모 특이 키 사지 헐렁 볼 폭 패 이상 광대뼈 창백 모습 입술 도톰 관능 때 머리 눈 비극 빛 손은 손가락 손은 비상 힘 인 사람 정열 인상 불안감 사람


In [13]:
make_keywords("""장 발장은 빵 한 조각을 훔친 죄로 총 19년간 감옥에서 복역하고 나온다. 그러고도 하룻밤 잠자리를 마련해 준 미리엘 주교의 집에서 은촛대를 훔쳐 달아난다. 그런데 장 발장이 헌병에게 체포되었을 때 미리엘 주교는 은촛대는 자신이 준 것이라고 말해 장 발장을 구한다. 덕분에 체포를 면한 장 발장은 비로소 사랑에 눈뜨고 마들렌이라는 새 이름으로 살면서 시장까지 되어 사람들의 존경을 받으며 새 삶을 살게 된다.
그러나 주교의 집에서 은촛대를 훔쳤을 당시 장 발장을 체포했던 경감은 끈질기게 그를 의심한다. 때마침 한 사나이가 장 발장으로 오인되어 체포되자 장 발장은 자수해 사나이를 구하고 감옥에 간다. 곧 탈옥한 장 발장은 지난날 자신이 도와주었던 여공을 찾아간다. 그녀는 죽기 직전에 자신의 딸 코제트를 장 발장에게 부탁한다. 장 발장은 다시 체포되었으나 곧바로 탈옥해 코제트를 데리고 파리로 도망친다. 그곳에서 장 발장은 열심히 일해 다시 성공한다. 그리고 코제트는 마리우스와 사랑에 빠져 결혼한다. 장 발장은 코제트 부부가 지켜보는 가운데 숨을 거둔다. 19세기 프랑스 대문호 빅토르 위고의 대표작!
가난한 전과자 장 발장을 통해 인간과 삶, 세상을 통찰한 독보적인 걸작!
19세기 프랑스의 대문호 빅토르 위고의 대표작을 꼽으라면 단연 《레 미제라블》이다. 이 작품은 역사, 사회, 철학, 종교, 인간사의 모든 것을 축적한 세기의 걸작으로 “한 저주받은 비천한 인간이 어떻게 성인이 되고, 어떻게 예수가 되고, 어떻게 하느님이 되는”지 그려 냈다. 빅토르 위고가 35년 동안 마음속에 품은 이이야기를 십칠 년에 걸쳐 완성한 이 작품은 워털루 전쟁, 왕정복고, 폭동이라는 19세기 격변을 다룬 역사 소설이자 당시 사람들의 애환을 그린 민중 소설이다. 사상가이자 시인 빅토르 위고의 철학과 서정이 담겨 그 자체로 “하나의 거대한 세계”나 다름없는 독보적인 걸작이다.
《레 미제라블》은 역사상 가장 유명한 소설 가운데 하나다. 하지만 이 작품을 ‘완독’한 사람은 의외로 많지 않을 것이다. 축약하거나 각색하지 않은 ‘무삭제판’ 《레 미제라블》을 처음 접한 사람은 두 번 놀라게 된다. 방대한 분량에 놀라고, 그 유명한 줄거리가 빙산의 일각에 불과하다는 사실에 또 한 번 놀란다. 장 발장에 대한 이야기는 이 소설에서 3분의 1가량 내용에 불과하며, 나머지 3분의 2에는 19세기 초의 프랑스 사회와 풍습, 그리고 다양한 문제에 관한 저자의 견해가 서술되어 있기 때문이다. 여기, 더클래식 세계문학컬렉션 《레 미제라블》의 전 구성으로 무삭제판 《레 미제라블》을 완독할 수 있다.

지옥을 그려 낸 단테가 있었듯, 나는 현실을 이용해 지옥을 만들어 내려 했다. _빅토르 위고

서사시와 소설, 그리고 에세이의 요소까지 갖추고 있는 걸작 _앙드레 모루와(문학비평가)

가장 위대한 아름다움이 자리한 이 소설은 하나의 세계이자 하나의 혼돈이다. _랑송(문학비평가)

20세기에 빅토르 위고와 견줄 만한 작가는 없다. 과연 21세기에도 그와 같은 작가가 나올지 의문이다. _해럴드 블룸(문학비평가)""")

장발장 빵 조각 죄 감옥 복역 하룻밤 잠자리 마련 미리엘주교 집 은 촛대 장발장 헌병 체포 때 미리엘주교 은 촛대 자신 말 장발장 덕분 체포 장발장 사랑 마들렌 이름 시장 사람 존경 삶 주교 집 은 촛대 당시 장발장 체포 경감 의심 사나이 장발장 오인 체포 장발장 자수 사나이 감옥 탈옥 한 장발장 지난날 자신 여공 직전 자신 딸 코제트 장발장 부탁 장발장 체포 탈옥 코제트 파리 장발장 일해 성공 코제트 마리우스와 사랑 결혼 장발장 코제트 부부 가운데 숨 프랑스 문호 빅토르위고 대표작 가난 전과자 장발장 인간 삶 세상 통찰 독보 걸작 프랑스 문호 빅토르위고 대표작 레미제라블 작품 역사 사회 철학 종교 인간사 축적 세기 걸작 저주 비천 인간 성인 예수 하느님 빅토르위고 동안 마음속 이야기 완성 작품 워털루 전쟁 왕정복고 폭동 격변 역사소설 이자 당시 사람 애환 민중 소설 사상가 시인 빅토르위고 철학 서정 자체 거대 세계 독보 걸작 레미제라블 역사 유명 소설 가운데 작품 완독 사람 축약 각색 삭제 판 레미제라블 처음 사람 방대 분량 유명 줄거리 빙산 일각 불과 사실 장발장 이야기 소설 내용 불과 나머지 프랑스 사회 풍습 다양 문제 저자 견해 서술 더 클래식 세계 문학 컬렉션 레미제라블 구성 삭제 레미제라블 완독 지옥 단테 현실 이용 지옥 빅토르위고 서사시 소설 에세이 요소 걸작 앙드레 모루 문학 비평가 위대 자리 소설 세계 혼돈 랑 송 문학 비평가 빅토르위고 작가 작가 의문 해럴드블룸 문학 비평가


In [14]:
make_keywords("""『남자의 향기』로 주목받은 작가의 두 번째 소설이다.
벼랑 끝에 피어난 한 송이 들꽃, 그 아무리 손 내밀어도 닿을 수 없는 여인을 위해 일생을 살아온 남자가 있다. 끝내 이루어질 수 없을 것 같은 남자의 사랑은 지구를 반 바퀴 돌아, 저 머나먼 땅 아프리카에서 새로운 전환점을 만나고야 만다. 안타까울 정도로 깊고 오랜 사랑을 차마 표현하지 못하는 남자와 그러한 남자를 곁에 두고도 항상 다른 사랑을 꿈꾸는 여자. 힘겨운 사랑 앞에 절망하고 있다면 이 남자의 사랑을 기억해야 한다. '희망 없는 사랑을 하는 자만이 사랑의 진정한 의미를 알고 있다 '는 실러의 말과 함께.""")

남자의향기 주목 작가 소설 벼랑 끝 송이 들꽃 손 여인 일생 남자 남자 사랑은 지구 반 바퀴 땅 아프리카 전환점 정도 사랑 표현 남자 그러 남자 곁 사랑 여자 사랑 앞 절망 남자 사랑 기억 희망 사랑 자만 사랑 진정 의미 실 러 말


In [15]:
make_keywords("""도산이 흥사단우를 고르는 표준은 두 가지 있었다. 첫째는 거짓이 없는 사람, 둘째는 조화성(調和性) 있는 사람이었다. 조화성이라 함은 단체 생활을 가능케 하는 성질이다. 너무 자기에 고집하고 규각(圭角)이 심한 사람은 단체 생활에 늘 말썽을 일으키기 때문이었다.

『거짓이 있는 사람, 규각을 세우는 사람이라도 한 가지 기술과 한 가지 능력이 있거든 받아서 수양을 시키면 좋지 아니하냐?』

하는 이론에 대하여서 도산은 이렇게 대답하였다.

『금주 동맹(禁酒同盟)은 술을 아니 먹는 사람들이 모임으로 성공하는 것이 다.』

고. 이것은 미국의 금주 운동이 카톨릭 신부로부터 시작한 것을 가리킴이었다.

거짓 없는 사람들이 많이 모여서 큰 단결을 이루면 그것이 거짓을 박멸하는 큰 힘을 내인다는 것이었다.

그러므로 도산은 흥사단우를 고를 때에 사회의 명서학식·수완 같은 것은 둘째로 여겼다. 「진실한 사람」이것이 첫 조건이었다. 학식이나 수완이 나라를 광복하고 민족을 구제하는 것이 아니라, 오직 진실만이 능히 한다고 도산은 믿은 것이었다. 그러나 명성·학식·수완이 있고 진실하면 금상 첨화(錦上添花)지마는 도산이 보기에 우리나라의 학식가·수완가는 반드시 진실을 존중하는 이가 아니었다. 소위 임기웅변(臨機雄辯)과 권모술수(權謀術數)를 진실보다 소중히 여기는 이가 많았다.

도산은 이것을 슬퍼하였다. 왜 그런고 하면, 이 때문에 지도자끼리나 또는 지도자와 민중이 서로 믿지 못하기 때문이었다. 학식은 배울 수도 있고 남에게 빌 수도 있고, 수완도 없으면 부족한 대로 하여 나갈 수가 있었다.

도산은 국가와 민족을 위하여서 영웅 호걸보다도 진실한 사람을 구하였다. 철두철미 거짓을 벗고 오직 참으로만 나가는 사람이야말로 나라를 구원하고 백성을 건지는 민족적 영우이라고 도산은 생각하였다.
... 책 속에서 ...""")

도산 흥사단 우 표준 거짓 사람 조화 사람 조화 단체 생활 가능 성질 고집 규 각 사람 단체 생활 말썽 거짓 사람 규 각 사람 기술 능력 수양 이론 도산 대답 금주 동맹 술 사람 모임 성공 미국 금주운동 카톨릭 신부 시작 거짓 사람 단결 거짓 박멸 힘 도산 흥사단 우 때 사회 서학 식수 진실 사람 조건 학식 수완 나라 광복 민족 구제 진실 도산 명성 학식 수완 진실 금상 첨 도산 우리나라 학식 가 수완가 진실 존중 임기 웅변 권모술수 진실 도산 고 지도자 지도자 민중 학식 남 빌 완도 부족 도산 국가 민족 영웅 호 걸 진실 사람 거짓 사람 나라 구원 백성 민족 영 우이 라고 도산 생각 책 속


In [16]:
make_keywords("""“사랑하고 배려하고 기다리는데 왜 날 떠나죠?”
해도 해도 늘지 않는 연애, 그것이 알고 싶다. 연애강자 나쁜 여자!
사랑받는 나쁜 여자가 되는 법과 알 수 없는 그 남자의 심리를 파헤치다

왜 남성들은 나쁜 여자에게 끌릴까? 독립적인 마인드로 경제적, 심적 부담을 주지 않을 것 같기 때문이라고 한다. 내숭 떨지 않고 감정 표현에 솔직한 모습, 커리어 및 외모 등 자기관리에 열심인 모습 등도 그 이유다.
이제 남성들도 독립적이고 주체적인 여성과의 동등한 연애를 바라는 욕망이 크다. 나쁜 여자가 내포하고 있는 당당함, 솔직함, 적극성, 독립성 등에 남성이 끌리고 있다. 전문직 남성일수록 더욱 나쁜 여자와의 연애에 호의적이다.
헌신하는 여자는 더 이상 매력적이지 않다. 쉽게 느껴지고 함부로 해도 된다는 인상을 남성에게 주기도 한다. 남자에게 헌신하는 여자는 헌 신짝이 된다고 하지 않는가. 그만큼 순종적이고 자기 희생적인 여자는 매력과 거리가 멀다고 느껴지는 시대다.
이 책은 여성들에게 ‘이기는 연애’를 위해 “착함을 버리고, 욕망한 것을 솔직하게 표현하라”라고 한다.
그렇다고 남자에게 어필하는 나쁜 여자에 초점을 맞추지 않았다.
나의 행복과 만족을 위해 먼저 이기적이고 나쁜 여자가 되었을 때 사랑도 얻을 수 있다는 점에서 자기애적이다. 늘 차이기만 하는 당신, 짝사랑만 하는 당신을 위한 책이다.
연애에 갓 입문한 당신, 해도 해도 연애가 늘지 않는 당신, 연애가 뭔지 모르는 모태 솔로에게도 추천한다.

줄거리
주인공 에리카는 몇 번의 연애를 경험하며 30대를 맞았다. 진정 사랑하는 사람을 찾기까지의 길은 주인공에게도, 그녀의 동성 혹은 이성 친구들에게도 어려운 일이긴 마찬가지다. 비슷한 고민 속에 살아가는 싱글, 혹은 커플인 친구들과 연애문제를 이야기하다가 에리카는 인상적인 일들을 기록한다.""")

사랑 배려 날 연애 그것이알고싶다 연애 강자 나쁜여자 사랑 여자가되는법 남자 심리 남성 나쁜여자 독립 마인드 경제 심 부담 내숭 감정 표현 솔직 모습 커리 어 외모 관리 열심 인 모습 유다 남성 독립 주체 여성 동등 연애 욕망 나쁜여자 내포 당당 솔직 적극 독립 남성 전문직 남성 나쁜여자 연애 호의 헌신 여자 이상 매력 인 남성 남자 헌신 여자 신짝 순종 희생 여자 매력 거리 시대 책 여성 연애 욕망 솔직 표현 남자 어필 나쁜여자 초점 행복 만족 이기 나쁜여자 때 사랑 자기애 당신 짝사랑 책 연애 입문 당신 연애 당신 연애 모태 솔로 추천 줄거리 주인공 에리카 연애 경험 진정 사랑 사람 길 주인공 동성 이성 친구 마찬가지 비슷 고민 속 싱글 커플 인 친구 연애 문제 이야기 에리카 인상 일 기록


In [17]:
make_keywords("""빈곤, 사회적 계급 관계 등 현실의 문제를 정면으로 다루면서 낭만주의를 벗어난 사실주의적 성격을 드러내는 소설 <행랑자식>, <자기를 찾기 전> 등의 단편 소설과 통속 소설의 취향을 따르면서 모호한 내면, 환상, 영탄을 사용하는 등 비극적 운명에 대한 감상주의가 드러난 <환희>등의 대표 작품을 모았다. 나도향은 낭만주의적 감상, 미학주의와 현실 비판의 냉정한 관찰력이 결합된 그의 소설작품을 통해 인간의 욕망, 내면을 중요시하는 낭만주의적인 것과 그런 욕망이 사회 속에서 드러내는 행태에 대한 객관적 묘사와 관찰을 동시에 보여주고 있다.""")

빈곤 사회 계급 관계 현실 문제 정면 낭만주의 사실주의 성격 소설 행랑 자식 전 단편 소설 통속 소설 취향 모호 내면 환상 영 탄 사용 비극 운명 감상주의 환희 등의 대표 작품 나도향 낭만주의 감상 미학 주의 현실 비판 냉정 관찰력 결합 소설 작품 인간 욕망 내면 중요시 낭만주의 욕망 사회 속 행태 객관 묘사 관찰 동시


In [18]:
make_keywords("""독일 바로크 시대의 첫 소설 작품. 30년 전쟁 당시 사회에서 일어나는 다양하고 변화무쌍한 이야기를 담았다. 양치기로 단순 무식하게 살아가던 소년이 전쟁에 휩쓸리면서 어떻게 변해 가는지를 보여 준다.""")

독일 바로크 시대 소설 작품 30년전쟁 당시 사회 다양 변화무쌍 이야기 양치기 단순 무식 소년 전쟁


In [19]:
make_keywords("""총1,062페이지에 걸쳐 210편 전편 수록!
<백설공주> <라푼첼> 등 유명 동화들의 원작 완역.

초판 발행 후 200년…
오늘날 더 많이 사랑받는 이야기.

일러스트의 거장 ‘아서 래컴’ 컬러 삽화 전체 수록.

<개구리 왕자> <백설공주> <라푼첼> <헨젤과 그레텔>. 어렸을 적 누구나 한 번쯤 들어봤을 만한 익숙한 동화의 제목이자 오늘날에는 애니메이션, 영화 등의 형태로 끊임없이 재해석되는 이야기들이다. 이 이야기들의 공통점은 독일의 유명한 학자이자 작가인 ‘그림 형제’가 약200년 전 수집했던 이야기들이 원작이라는 점이다. 그림 형제는 유럽 지역에 전해 내려오던 이야기들 속에서 인간적인 심성의 기원이 무엇인지를 밝히고자 노력했고, 이들의 노력으로 이 이야기들은 시간이 지날수록 더 많은 사람들에게 사랑받고 있다.
이 책에 담긴 총210편의 원작(Original)을 통해 인간 본성의 여러 가지 모습과 함께 색다른 재미를 느낄 수 있다.""")

페이지 전편 수록 백설공주 라푼첼 유명 동화 원작 완 역 초판 발행 후 오늘날 사랑 이야기 러스트 거장 아서래컴 컬러 삽화 전체 수록 개구리 왕자 백설공주 라푼첼 헨젤과그레텔 익숙 동화 제목 오늘날 애니메이션 영화 형태 해석 이야기 이야기 공통점 독일 유명 학자 작가 인 그림형제 수집 이야기 원작 그림형제 유럽 지역 전해 이야기 속 인 간적 인 심성 기원 인지 노력 노력 이야기 시간 사람 사랑 책 원작 인간본성 모습 재미


In [20]:
make_keywords("""나쓰메 소세키가 100년 전에 움켜쥐고 고민한, 지금도 유효한 물음

나쓰메 소세키가 문학과 학문을 통해 끊임없이 질문을 던지고 답하고자 천착한 것은 ‘이 시대를 살아가는 우리에게 인간적으로 산다는 것이 무엇인가’라는 근본적인 문제이며 이는 곰곰이 생각해볼 인생의 화두가 된다.

그중 2차분 네 권(『우미인초』, 『갱부』, 『산시로』, 『그 후』)에서는 불안과 불만으로 “바싹 말라버린 청춘”을 사유하도록 이끈다. 삶과 죽음, 사랑과 고통 등 청춘이 마땅히 누려야 할 ‘발랄’과는 거리가 먼, 번민만이 흩어져 있던 “불행한 시대”의 100여 년 전 이야기는 일본이라는 공간을 넘고 시대를 넘어 지금, 우리에게도 유효하다.

100년 동안 수없이 많은 독자가 가슴속에 간직해온 ‘살아 있는’ 소세키를 읽을 수 있도록 고심해서 각 권 마지막에 우리 문학가들의 ‘소세키 독후감’을 담았다. 소설가 강영숙이 읽은 우미인초의 자줏빛 ‘봄날의 산행’, 소설가 장정일이 말하는 『갱부』로 거듭나기, 소설가 김연수가 담은 『산시로』의 잃어버린 청춘의 한 조각, 시인 김경주가 찾은 『그 후』의 그윽한 문장들… 다양한 분야에서 활발하게 활동하고 있는 우리 작가들이 그들만의 소세키를 ‘해설 아닌 해설’의 자유로운 형식으로 담아 한국 독자들의 소세키 읽기에 즐거움을 더했다.""")

나쓰메소세키 전 고민 지금 유효 물음 나쓰메소세키 문학 학문 질문 천착 시대 인간 근본 문제 생각 인 화두 그중 권 우미인초 갱 부 산시로 그후 불안 불만 청춘 사유 삶 죽음 사랑 고통 청춘 발랄 거리 번민 불행 시대 이야기 일본 공간 시대 지금 유효 동안 독자 가슴속 간직 소세키 고심 마지막 문학가 소세키 독후감 소설가 강영숙 우미인초 자줏빛 봄날 산행 소설가 장정일 말 갱 부 소설가 김연수 산시로 잃어버린청춘 조각 시인 김경주 그후 그윽 문장 다양 분야 활발 활동 작가 소세키 해설 해설 자유 형식 한국 독자 소세키 읽기 즐거움


In [21]:
make_keywords("""세상과의 경계에 서 있는 젊음의 불안과 방황을 통한 자아실현과 영적 탐구를 헤르만 헤세만큼 투명하고 생생하게 보여준 작가는 없었다. 현대문학에서는 질풍노도의 성장기에 겪었던 혼돈과 투쟁, 그리고 그것을 통해 완전한 자유에 이르는 과정을 기록한 헤세의 날카롭고 섬세한 글들을 「헤르만 헤세 선집」으로 묶어 소개한다. 선과 악, 밝은 세계와 어두운 세계, 자연과 정신, 육체와 영혼의 이분법을 지양하고, 삶에 대한 더 높은 지평을 제공하는 헤세의 작품들은 험난한 이 세상을 이해하는 데 좋은 동반자가 될 것이다.

『게르트루트』는 음악이라는 소재를 전면에 내세운 헤세의 초기작이다. 국내에는 1970년대에 『사랑의 삼중주』라는 제목으로 소개되었는데 그당시 제목대로 '게르트루트'는 사랑의 삼각관계를 다룬 작품으로 읽을 수도 있고 헤세의 보편적 주제인 양극성의 조화에 대한 음악적 변주로 볼 수도 있다. 헤세가 자신의 자아를 두 인물로 분리하여 그린 수많은 작품들처럼 『게르트루트』에서도 일인칭 화자인 작곡가 쿤과 삼인칭으로 묘사되는 오페라 가수 무오트는 한 예술가의 아폴론적인 속성과 디오니소스적인 속성을 형상화한 것이다. 헤세 소설의 주인공들은 대부분 이원적인 존재이다. 그들은 인간이면서 늑대이고, 범죄자이면서 신사이고, 소시민이면서 예술가이고, 건강하면서 병들어 있다. ‘내 가슴에는 두 영혼이 깃들어 있다’는 괴테의 『파우스트』의 다양한 변주라고 할 수 있다.""")

세상 경계 젊음 불안 방황 자아실현 영 탐구 헤르만헤세 투명 생생 작가 현대문학 질풍노도 성장기 혼돈 투쟁 완전 자유 과정 기록 헤세 섬세 글 헤르만헤세 선집 소개 선 악 세계 세계 자연 정신 육체 영혼 이분법 지양 삶 지평 제공 헤세 작품 험난 세상 이해 동반자 게르트루트는 음악 소재 전면 헤세 초기작 국내 사랑 삼중주 제목 소개 당시 제목 게르트루트는 사랑 삼각관계 작품 헤세 보편 주제 인 양극 조화 음악 변주 헤세 자신 자아 인물 분리 작품 게르트루트에서도 일인칭 화자 인 작곡가 쿤 삼인칭 묘사 오페라 가수 무오 예술가 아폴론 속성 디오니소스 속성 형상화 헤세 소설 주인공 대부분 이원 존재 인간 늑대 범죄자 신사 소시민 예술가 건강 가슴 영혼 있다 괴테 파우스트 다양 변주


In [22]:
make_keywords("""“백조파 특유의,
감상적이고 환상적인 작품”
20세기 한국 문학을 대표하는 작가들의 시와 단편 소설 작품들을 엄선한 소설 선집 <한국문학읽다> 는 문학의 고전을 살아 있는 동시대의 문학으로 읽을 수 있도록 구성한 시리즈이다.한국인이 꼭 읽어야 할 대표 작가들의 주요 작품을 고전부터 근ㆍ현대 작품에 이르기까지 우리나라 대표 작가들의 다양한 작품을 만날 수 있다.<나도향 단편문학>은 <노천명 단편문학>, <이효석 단편문학>, <방정환 단편문학>, <윤동주 시집>, <김소월 시집>, <김동인 단편문학>에 이은 <한국 문학 읽다>시리즈 7번째 작품이다.
""")

백조 파 특유 감상 환상 작품 한국문학 대표 작가 시와 단편 소설 작품 엄선 소설 선집 한국 문학 문학 고전 동시대 문학 구성 시리즈 한국 인 대표 작가 주요 작품 고전 근현대 작품 우리나라 대표 작가 다양 작품 나도향 단편 문학 노천명 단편 문학 이효석 단편 문학 방정환 단편 문학 윤동주 시집 김소월 시집 김동인 단편 문학 한국문학 시리즈 작품


In [23]:
make_keywords("""이 책속의 내용을 보면 막내왕녀공주가 황금공을 갖고 놀다가 공을 그만 깊은 샘물 웅덩이에 빠트리게됩니다.

공주는 공을 못찾을 생각에 슬피 울고 있었습니다. 그런 공주에게 험상궂은 개구리가 나타나서 황금공을 공주님께 찾아주면 저에게 무엇을 줄터입니다까? 하고 물었다.

그때 개구리가 원하는 소원을 말하였고, 공주는 개구리의 소원을 들어주기로하고 사라져 버립니다...

공주와 개구리는 어떻게 될까요?
...

이 책은 아이들 눈높이에 맞는 그림과 내용으로 읽기 좋은 동화이며, 또한 아이들이 삶에 교훈을 배울수 있어서 더좋은 책입니다.""")

책 속 내용 막내 왕녀 공주 황금 공 공 샘물 웅덩이 공주 공 생각 공주 개구리 황금 공 공주 그때 개구리 소원 말 공주 개구리 소원 공주와개구리 책 아이들 눈높이 그림 내용 읽기 동화 아이들 삶 교훈 책


In [24]:
make_keywords("""5회 김만중문학상 금상 수상작. 조완선의 장편소설로, 조선시대 최고의 두 문장가 교산 허균과 연암 박지원이 '홍길동전'과 '허생전'이라는 두 걸작을 탄생시킨 과정을 소설적 상상력을 통해 흥미진진하게 보여준다.

1618년 역모죄로 체포돼 사지가 찢기는 처참한 최후를 맞은 교산 허균. 그의 책은 왕이 일곱 번 바뀌도록 금서로 묶여 있다. 남몰래 교산의 책을 접하며 그를 흠모해오던 연암 박지원에게 어느 날 책쾌 조열이 찾아와 그동안 알려지지 않았던 교산의 책을 구해 오겠다고 한다. 한껏 기대에 부풀어 있던 연암은 약속한 날짜를 한참 넘겨서도 소식이 없는 조열을 찾아 나섰다가 그가 살해되었다는 비보를 접하고, 그가 죽음에 이른 곡절을 알아내고자 먼 길을 떠난다.

문제의 책은 교산이 홍길동의 마지막 행적을 추적하며 겪은 일을 기록한 '교산기행'. 이 소설에서는 홍길동의 자취를 밟아가는 교산의 험난한 여정과 조열이 죽은 연유를 밝히려는 연암의 긴박한 여행길이 168년의 시간을 넘나들며 진진하게 교차된다. 홍길동은 어디로 사라졌을까? 조열을 죽인 자는 누구이고, 교산의 서책은 어디에 있을까?""")

김만중 문학상 금상 수상작 조 완선 장편소설 조선 시대 최고 문장가 교산 허균 연암 박지원 홍길동전 허생전 걸작 탄생 과정 소설 상상력 흥미진진 역모 죄 체포 사지 처참 최후 교산 허균 책 왕이 금서 교산 책 흠모 연암 박지원 날 책 쾌 그동안 교산 책 기대 연암 약속 날짜 한참 소식 살해 비보 죽음 곡절 길 문제 책 교산 홍길동 마지막 행적 추적 일 기록 교산 기행 소설 홍길동 자취 교산 험난 여 정 연유 연암 긴박 여행길 시간 진진 교차 홍길동 교산 서책


In [25]:
make_keywords("""억압과 구속과 자기혐오가 만연한
평화로운 다이어트랜드에 나타난 정체 모를 "여성 테러리스트"들,
여성을 향한 폭력과의 전쟁을 선포하다!

얼떨결에 이상한 나라의 페미니스트 공동체에 떨어져
여성 해방 전쟁의 최전방에 서게 된
"뚱보" 플럼 케틀의 다이어트랜드 탈출기

평생 뚱뚱한 몸 때문에 수치심과 자기혐오에 시달려온 플럼 케틀은 비만수술을 받아 날씬하고 예쁜 여자로 거듭날 그날만을 기다리고 있다. 그런 그녀 앞에 알록달록한 스타킹을 신은 정체 모를 여성이 나타난다. 그 여성에게 이끌려 페미니스트 공동체 "칼리오페 하우스"에 발을 들이게 되면서 플럼의 계획이, 그녀의 삶이 송두리째 흔들리기 시작한다. 타인의 시선을 통해 자신을 바라보는 것에 익숙해져버린, 자신의 몸에 대한 애정과 소유권을 잃어버린 모든 여성들을 향한 반(反)다이어트 선언문!""")

억압 구속 자기혐오 만연 평화 다이어트 랜드 정체 여성 테러리스트 여성 폭력 전쟁 선포 얼떨결 이상 나라 페미니스트 공동체 여성 해방 전쟁 최 전방 뚱보 플럼 틀 다이어트 랜드 탈출기 평생 뚱뚱 몸 수치심 자기혐오 플럼 비만 수술 날씬 여자 그날 앞 스타킹 신 정체 여성 여성 페미니스트 공동체 칼리오페 하우스 발 플럼의 계획 삶 송두리 시작 타인 시선 자신 익숙 자신 몸 애정 소유권 여성 다이어트 선언문


In [26]:
make_keywords("""『노예 12년』은 뉴욕 주의 자유 시민인 솔로몬 노섭이 자유를 뺏기고 노예가 되어서 12년이 지나 다시 자유를 되찾기까지의 여정을 담은 실화이다. 19세기 후반 미국 역사의 어두운 부분을 들추고 인간에게 인권과 자유란 무엇인지 화두를 던진다는 점에서 출간 후 3년간 3만 부가 팔리며 베스트셀러가 되었다. 이후 1968년 루이지애나의 역사학자인 수 아이킨과 조지프 로그즈던이 솔로몬 노섭의 행적을 추적해 거의 모든 장소와 인물들, 기록 등의 실재를 밝혀내는 고증 작업을 거치면서 다시금 주목받기 시작했다. 1984년에는 〈솔로몬 노섭의 오디세이〉라는 이름으로 PBS 텔레비전 영화로 만들어졌고, 1999년부터는 솔로몬 노섭이 살던 곳인 새러토가스프링스에서는 7월 셋 째 주 토요일을 〈솔로몬 노섭의 날〉로 정해 기념해 오고 있다. 2013년에는 스티븐 매퀸 감독의 영화로 제작되어 아카데미, 골든글로브, 런던 비평가 협회에서 다수의 수상을 하며 세간의 주목을 받았다.

『노예 12년』이 출간된 당시, 해리엇 비처 스토의 소설 『톰 아저씨의 오두막』을 통해 노예 제도의 폐해에 관심이 집중된 만큼 『노예 12년』은 실화를 담았다는 점에서 보다 큰 사회적 반향을 불러일으켰다. 솔로몬 노섭은 12년 동안 몸소 겪은 노예 생활을 통해 자유를 위해 투쟁할 수밖에 없었던 그 시대 노예들의 심정과 생생한 삶의 장면을 솔직하게 묘사했다.

『노예 12년』은 작가 자신이 납치당해 노예로 12년 동안 살게 된 극적인 사건을 통해 당시 노예 제도의 폐해와 어두운 그림자를 여실히 보여 준다. 노예 수용소, 벌목지, 목화밭, 사탕수수 밭 등을 전전하며 여러 노예와 주인 들을 만난 솔로몬 노섭은 노예 제도의 현실을 정확하게 그려 내고 선입관과 편견에 치우치지 않으려 노력하며 편집자인 데이비드 윌슨의 도움을 받아 이 책을 출간했다. 고증을 통해 밝혀졌듯, 이 책은 실화를 기록했다는 점, 노예 제도를 사실성에 입각해 그대로 담아냈다는 점에서 문학뿐만 아니라 사료로도 그 의미가 깊다. 1976년에는 알렉스 헤일리의 『뿌리』와 함께 노예 제도와 미국 흑인의 역사에 대한 필독서로 선정되어 교재 등으로 널리 읽히기도 했다.

이 책은 일대기에 갇히지 않고 시대상을 객관적으로 바라보고 기술했다는 점에서 기존의 노예 서사와 다르다. 태어날 때부터 자유를 갖고 있었으며 다양한 교육을 받고 자유인으로서 사고하던 흑인이 자유를 빼앗긴 뒤 노예의 삶을 경험했다는 측면에서 그가 옮겨간 미국 남부의 자연과 특성, 농법, 노예 제도에 관해 더욱 세밀히 관찰하고 객관적으로 이야기할 수 있었다. 이 책에는 단순히 착취당하고 억압받는 노예의 수난만 다룬 게 아니다. 당시 미국 사회의 단면들을 보여 주는 풍부한 소재와 묘사들이 넘친다. 솔로몬 노섭 스스로 백인의 정신을 가지고 살았다고 인정할 정도로, 그는 항상 자신이 자유인임을 의식하고 늘 깨어 있었고, 자신의 과거를 숨겨야만 하는 상황에서도 정체성을 잃지 않으려 부단히 노력했다.

19세기 후반부터 20세기를 넘어 노예 제도가 사라진 21세기까지 『노예 12년』이 주목받는 이유는 작품이 갖는 역사적 의미와, 시대를 관통해 현재까지 여전히 유효한 〈인간다움〉의 의미를 묻고 있기 때문이다. 노예 제도는 이미 폐지되었고 미국에서는 최초의 흑인 대통령이 탄생했지만 아직까지도 전 세계적으로 인권 유린 문제는 잔존하고 있다. 이 작품은 우리 시대에 인간과 비인간의 경계에 서서 진정한 〈인간다움〉과 〈정의〉가 무엇인지 다시금 되돌아보게끔 한다.""")

노예12년 뉴욕주 자유 시민 인 솔로몬노섭 자유 노예 자유 여 정 실화 후반 미국 역사 부분 인간 인 자유 화두 출간 후 부가 베스트셀러 이후 루이지애나 역사학자 인 아이 지프 로그즈던이 솔로몬노섭 행적 추적 장소 인물 기록 실재 고증 작업 주목 시작 솔로몬노섭 오디세이 이름 텔레비전영화 솔로몬노섭 곳 인 새러토가스프링스 토요일 솔로몬노섭 날 정 기념 스티븐 퀸 감독 영화로 제작 아카데미 골든 글로브 런던 비평가 협회 다수 수상 세간 주목 노예12년 출간 당시 해리엇비처스토 소설 톰아저씨의오두막 노예 제도 폐해 관심 집중 노예12년 실화 사회 반향 솔로몬노섭 동안 노예 생활 자유 투쟁 시대 노예 심정 생생 삶 장면 솔직 묘사 노예12년 작가 자신 납치 노예 동안 극 사건 당시 노예 제도 폐해 그림자 노예 수용소 벌목 목화 밭 사탕수수 밭 전전 노예 주인 솔로몬노섭 노예 제도 현실 정확 선입관 편견 노력 편집자 인 데이비드윌슨 도움 책 출간 고증 책 실화 기록 노예 제도 사실 입각 문학 사료 의미 알렉스 헤일리 뿌리 노예 제도 미국 흑인 역사 필독서 선정 교재 책 일대기 시대상 객관 기술 기존 노예 서사 때 자유 다양 교육 자유 인 사고 흑인 자유 뒤 노예 삶 경험 측면 미국남부 자연 특성 농법 노예 제도 관찰 객관 이야기 책 착취 억압 노예 수난 게 당시 미국 사회 단면 풍부 소재 묘사 솔로몬노섭 백인 정신 인 정 정 도로 자신 자유 인 의식 자신 과거 상황 정체성 노력 후반 노예 제도 노예12년 주목 이유 작품 역사 의미 시대 관통 현재 유효 인간 의미 노예 제도 폐지 미국 최초 흑인 대통령 탄생 세계 인 유린 문제 잔존 작품 시대 인간 인간 경계 서서 진정 인간 정의


In [27]:
make_keywords("""스페이스 오페라의 고전
여성 작가의 호쾌한 우주 전쟁 서사시

SF 역사상 최고 걸작 중 하나이자 [스페이스 오페라] 장르의 대표작인 C. J. 체리의 『다운빌로 스테이션』이 최용준 번역으로 열린책들에서 출간되었다. 국내 초역. 체리는 당시 SF계에 희귀했던 여성작가로서, 이 책으로 휴고상을 수상했다. 『다운빌로 스테이션』은 27권의 장편으로 이루어진 장대한 [유니언-동맹 소설]의 첫 권이자 체리의 대표작이다. 우주가 개발됨에 따라 필연적으로 지구의 통제력이 약화되고, 지구 세력과 신세계(우주) 세력 간의 갈등이 기나긴 전쟁으로 비화되는 과정과, 그 전쟁이 어떤 결말을 맞이하는지 그리고 있다. 천문학과 우주 공학을 전공한 번역가 최용준 씨는 한국을 대표하는 SF 전문 번역가로서, 복잡하고 방대하며 까다롭기로 정평이 있는 이 고전의 신뢰성 있는 한국어판을 제공하기 위해 여러 해 동안 심혈을 기울였다.""")

스페이스오페라 고전 여성 작가 호쾌 우주전쟁 서사시 역사 최고 걸작 이자 스페이스오페라 장르 대표작 인 체리 다운 빌 스테이션 최용준 번역 열린책들 출간 국내 역 체리 당시 계 희귀 여성 작가 책 휴고상 수상 다운 빌 스테이션 장편 장대 유니언 동맹 소설 체리 대표작 우주 개발 필연 지구의 통제력 약화 지구 세력 신세계 우주 세력 갈등 전쟁 비화 과정 전쟁 결말 맞이 천문학 우주 공학 전공 번역가 최용준 한국 대표 전문 번역가 복잡 방대 정평 고전 신뢰 한국어 판 제공 동안 심혈


In [28]:
make_keywords("""“누군가를 좋아할 때는 내 몸에서도 빛이 나.

좋아하는 사람으로 인해 자신 또한 빛나는 거지.”



그렇게 팔 년 동안 나는 그치지 않고 빛났다.



설렘을 간직한 첫사랑을 그린 영화로 잘 알려진 대만 영화 [그 시절, 우리가 좋아했던 소녀](2011)의 동명 원작 소설이 문학동네에서 출간되었다.



영화 [그 시절, 우리가 좋아했던 소녀]는 [말할수 없는 비밀](2007)에서 시작해 [나의 소녀시대](2015)로 이어지는 ‘대만 청춘영화’의 계보에 속한다. 교복 입은 학생들이 주인공으로 등장한다는 점, ‘청춘’ ‘꿈’ ‘희망’ ‘성장’ ‘사랑’ 등을 키워드로 추억을 소환하는―시간과 기억에 관한 영화라는 점에서 이들 ‘대만 청춘영화’의 공통점을 찾을 수 있다. 흥행 면에서는, 대만 영화 흥행의 신호탄이 되었던 [말할 수 없는 비밀](16만여 명)이나, [말할 수 없는 비밀]이 8년간 지키고 있던 기록을 단숨에 갈아치운 [나의 소녀시대](41만여 명)에 못 미치는 기록을 남겼지만 [그 시절, 우리가 좋아했던 소녀]는 유독 ‘첫사랑’ 영화로서 많은 영화 팬들에게 그 존재감이 강하게 각인되어 있다. 이런 존재감을 증명이라도 하듯 국내에서는 2012년 8월 처음 개봉했던 영화가 4년이 지난 2016년 12월에 재개봉되기도 했고, 일본에서는 2018년 개봉 예정으로 리메이크 제작중이다.



영화 [그 시절, 우리가 좋아했던 소녀]가 이처럼 2010년대 ‘첫사랑 서사’의 대표적인 스토리로 자리잡게 된 가장 큰 힘은, 각본을 쓰고 연출한 주바다오 감독의 자전적 이야기를 바탕으로 한 실화 작품이라는 데 있다.



중고등학교부터 대학 시절까지 주바다오 감독 자신의 8년에 걸친 청춘에 대한 기록으로, 첫사랑의 풋풋함과 알싸한 상처를 입체적으로 그려낸 소설 『그 시절, 우리가 좋아했던 소녀』는 2005년 잡지 [HERE]에 연재되면서 큰 인기를 얻어 2011년 7월 단행본으로 출간되었고, 작가 주바다오가 직접 각본과 감독을 맡은 동명 영화가 뒤이어 개봉되었다. 8월 개봉한 영화는 2011년 대만 전체 박스오피스 3위를 기록했고, 5년 연속 홍콩에서 중화권 영화 박스오피스 1위 자리를 지켰으며, 제13회 타이베이 영화제(2011) 관객상, 제31회 홍콩 영화제(2012) 대만/중국 지역 최우수영화상, 제12회 중국영화미디어상(2012) 신인감독상과 작품상을 수상했다. 소녀, 우리가 좋아했던
― 고개를 돌리면 소녀가 맑다 못해 반짝 빛나는 눈망울로 나를 쳐다보았다

영화와 소설 모두 소년 ‘커징텅’과 소녀 ‘선자이’가 이야기의 중심을 이루지만, 소설 『그 시절, 우리가 좋아했던 소녀』는 영화의 모태가 된 ‘원작’답게 한층 깊고, 넓고, 입체적이다. 영화라는 매체적 특성상 고등학교 시절을 중심으로 축약될 수밖에 없었던 ‘사건 시간’이 소설 원작에서는 중학생 시절부터 서른두 살 성인이 되고 난 이후까지 한층 넓은 생애 시간을 망라하면서도, 사건과 심리 모든 면에서 촘촘한 묘사의 밀도를 유지하고 있다. 3인칭 관찰자 시점을 취할 수밖에 없었던 영화와 달리 서른두 살 커징텅의 1인칭 주인공 시점으로 과거와 현재를 넘나들며 서술되는 원작 소설에서 두 주인공 ‘커징텅’과 ‘선자이’ 캐릭터는 보다 입체적이고, 인물들의 선택과 행동 이면에 놓인 심경을 향하는 작품의 시선은 한층 깊고 섬세하다. 그리고 영화와 원작 소설의 가장 큰 차이라면, 원작 『그 시절, 우리가 좋아했던 소녀』에는 선자이 말고 한 명의 여학생이 더 등장한다는 점이다.

*

“그럼 우리 이야기를 쓰면 되겠네.” 군모를 쓴 랴오잉훙이 웃었다.
“그거 좋네, 우리 얘기를 글로 남겨봐.” 미국에서 유학중인 쉬보춘이 인터넷 게시판에 글을 남겼다.
그들의 말에, 내 등에서 날개 한 쌍이 소리 없이 자라나왔다.
“좀더 생각해보고.” 나는 고개를 저었다. 아직 바람이 일지 않았으니까.
그리고, 그녀에게서 전화가 걸려왔다…… (본문 9쪽)

오프라인 잡지와 온라인 사이트에 동시다발적으로 여러 작품을 발표하거나 연재하며 활발하게 작품 활동을 이어가고 있던 소설가 커징텅은 최근 반년 사이 몸져누우신 어머니 병상을 지키면서 뜻밖에 ‘사색의 시간’을 갖게 된다. 병상 곁에서 기억의 문을 열고 어머니에 관한, 어리고 철없던 자신에 관한 모든 것을 하나씩 세밀하게 기록해가던 그에게 어느 날 ‘청춘’이라는 단어로 시작되는 홍수가 밀려온다. 마침 새로운 작품 소재를 고민하고 있던 그는, ‘청춘’을 떠올릴 때 떼려야 뗄 수 없는 친구들의 조언, 그리고 결정적으로 그 청춘을 항상 빛나게 만들어주던 ‘그녀’에게서 걸려온 전화 한 통을 계기로 결국 새로운 이야기를 쓰기 시작한다. ‘그 시절’의 이야기를, 한 ‘소녀’를 함께 좋아하던 ‘우리’에 관하여.

소설 속 커징텅이 쓰는 ‘그 시절’에 대한 소설은 1990년 여름, 장화 현 징청 중학교 2학년 미술 A반 교실에서 시작된다. 수업 시간이든 쉬는 시간이든 가리지 않고 떠들며 장난치기를 좋아하는 소년 커징텅은 연습장에 연재 만화를 그려 반 친구들과 돌려 읽는 게 취미이고, 장차 만화가가 될 거라는 막연한 믿음을 가지고 있을 뿐, 성적이나 공부는 전혀 관심 밖의 일이다. 그러던 그의 일상에 잔잔한 파문과도 같은 변화가 일기 시작한다. 수업 시간에 한시도 가만히 있지 못하는 커징텅을 보다 못한 담임 선생님이 우등생이자 모범생인 선자이의 바로 앞으로 자리를 옮기게 하신 것. 그러고는 “이 골칫거리를 너한테 좀 부탁하자”는 말씀까지. 선자이는 마지못해 “네”라고 대답하는 듯했지만, 어느새 커징텅과 일상 ‘수다’를 허물없이 주고받으며 “약도 없는 유치함”의 소유자 커징텅에게 “노인네”처럼 잔소리를 끊임없이 늘어놓는다.

고개를 돌리면 선자이가 맑다 못해 반짝 빛나는 눈망울로 시선도 피하지 않고 나를 쳐다보았다.
(……) “너 머리 좋아 보여. 조금만 더 열심히 공부하면 성적이 꽤 오를 것 같아.” 선자이가 덤덤하게 말했다. (……)
그렇게 우리는 이야기를 나누기 시작했다. ‘커징텅의 인생에는 교정이 필요하다’는 뭐 그런 유의 이야기를. (본문 21~22쪽)

잔소리와 수다를 즐기는 선자이였지만 뜻밖에도 우등생 특유의 거들먹거림은 없었다. 자기 입으로 공부를 잘한다는 이야기를 꺼낸 적도 없고, 묘하게 사람 마음을 편안하게 해주는 구석이 있었다. 그리고 어느 날인가부터 펜끝으로 등을 쿡 찌르는 느낌에 돌아보면 선자이가 이 과목, 저 과목 교과서를 내놓으라고 한다. 그렇게 건네진 교과서는 온통 색색의 형광펜 밑줄과 요점 정리 메모가 빼곡히 적힌 채 되돌아온다. 그런 선자이의 싫지 않은 잔소리와 참견 덕분에 커징텅은 성적이 조금씩 오르기 시작하고, 학교의 새로운 분반 정책으로 시험 성적에 따라 A반을 떠나게 될지도 모른다는 불안 속에 치른 시험에서 합격선 안에 들어 A반에 남을 수 있게 된다. 이제 커징텅이 마음을 잡았다고 생각한 선생님은 커징텅의 자리를 재배정해주는데, 이번에도 ‘모범생’ 리샤오화의 뒷자리이다.
우연히 커징텅의 수학 점수가 자신보다 더 높게 나온 시험지를 본 리샤오화는 커징텅에게 자신이 해결하지 못한 수학 문제들을 물어보고, 실상은 선자이 덕분에 가까스로 받게 된 수학 점수였음에도, 자신을 ‘수학 잘하는 아이’쯤으로 생각하는 리샤오화 앞에서 망신당하지 않기 위해 커징텅은 기를 쓰고 밤마다 수학 공부에 몰두하면서 ‘노력’이라는 것에 대해서도 새로이 생각해보게 된다.

사람은 자신의 능력을 너무 높이 평가하면 안 된다. 그건 ‘노력’이라는 두 글자가 마땅히 누려야 할 영광을 빼앗는 셈이다. 내 청춘에서 만난 두 여학생이 바로 이 점을 나에게 알려주었다…… 그리고 노력의 끝에는 아름다운 풍경이 기다리고 있다는 사실을 굳게 믿도록 해주었다. 쉬지 않고 계속해서 노력하면, 믿기 힘든 놀라운 세상을 만나게 된다.
레드리스트의 상장을 받아들고, 자리로 돌아왔다.
“멋진데. 머리 좋아서 정말 좋겠다.” 리샤오화가 내 쪽으로 고개를 돌렸다.
“아…… 아니야……” 나의 그 출처를 알 수 없던 자신감이 자취를 쏙 감춰버렸다.

너로 인해. (본문 70쪽)

앞뒤 자리에서 함께하는 시간이 많아지고, 하교길에는 나란히 자전거를 끌고 걸어가며 손을 잡을까 말까 하는 망설임에 가슴 졸일 만큼 리샤오화와 각별한 사이가 되지만, 함께 어울려 다니던 동성 친구들 무리와의 우정과 커징텅 사이에서 고민하던 리샤오화는 결국 친구들 사이에서의 소속감을 택하는 대신 커징텅에게 싸늘하게 등을 돌리고 만다. 그리고 고입 연합고사가 끝난 뒤, 커징텅과 몰려다니며 짓궂은 장난을 도모하던 친구들은 물론 선자이까지 징청 중학교와 한 울타리에 있던 징청 고등학교로 진학하지만, 리샤오화는 장화 여고를 선택한다. 커징텅은 이미 “너는 머리가 좋으니까, 자연계에 진학하면 딱이겠다”라는 리샤오화의 한마디 때문에, 관심도 없고 적성에 맞는지도 모르던 자연계로 진학한 마당에. 그렇게 추억의 한자락이 막을 내린다.

장화 여고로 진학한 여학생들을 제외하고 미술 A반 친구들은 중학교와 같은 교정에 있는, 같은 학교로 거의 함께 올라가게 되고, 심지어 담임 선생님까지 그대로 고등학교로 올라오시는 바람에, 새로울 것이라고는 없는 심상한 날들이 이어진다. 다만 이 심상한 날들 가운데 단 한 가지 달라진 ‘특별함’이 있다면, 바로 선자이였다.

어쨌든, 선자이와 나 두 사람의 에너지는 줄곧 플러스마이너스 ‘중화’ 상태를 유지했다. (본문 22쪽)

담임 선생님이 내리신 벌로 처음 앞뒤로 나란히 앉게 된 이래 커징텅과 선자이 사이의 에너지는 한동안 덤덤하고도 담백하게 유지되어왔다. 선자이는 리샤오화를 두고 속앓이를 하는 커징텅의 도움 요청에 손을 내밀어주기도 했다. 고등학생이 된 어느 날, 수업이 끝났는데도 집에 가지 않고, 비어 있는 중학교 교실에 들어가는 선자이의 모습을 보게 되면서부터, 둘 사이 ‘중화’ 상태의 균형에는 의미 있는 균열이 생기기 시작한다.
매일 방과 후 학교에 남아 공부하는 선자이를 따라 커징텅도 함께 남아 공부를 한다. 다시 티격태격 중학교 때처럼 장난스레 싸우다 장난스레 일상 수다를 공유하고, 장난스레 놀리다 장난스레 함께 공부하는 날들이 이어진다. 선자이 덕분에 조금씩 성적이 올라가자 나름의 성과에 재미와 보람을 느낀 커징텅은 선자이에게 성적 내기를 제안한다. 돌아오는 시험에서 국영수 세 과목만의 점수를 합해, 커징텅이 이기면 선자이가 포니테일로 머리를 묶고 다니고, 선자이가 이기면 커징텅이 머리를 삭발하기로 한 것. 결과는 선자이의 승. 커징텅은 장대비를 뚫고 자전거를 타고 나가 머리를 바짝 밀고 돌아와서는, 빈 교실에 남아 공부중이던 선자이를 향해 찡긋 웃어 보이고, 선자이도 그 모습에 웃음을 터뜨린다. 그리고 며칠 뒤, 커징텅은 아무 내색 없이 포니테일로 머리를 묶고 나타난 선자이를 보고 마음 한편이 저릿하다.

사랑은 한 사람을 평소와 다른 모습으로 변화시켜준다. 그게 아니라면, 사랑이 가진 매력도 별것 아니지 않을까…… 우리가 밤낮으로 기도하며 바라는, 사랑이라 불릴 자격이 충분한 그런 사랑이 아니지 않을까. (본문 183쪽)

*

대학입학 시험 날. 선자이는 커징텅이 직접 그림을 그려 졸업 선물로 준 티셔츠를 입고 시험장에 나타난다. 사과를 품은 눈동자 그림이 그려진 티셔츠와 함께 커징텅이 전한 편지에는 이렇게 쓰여 있었다. “You are the apple of my eye.”
하지만 선자이는 시험을 망쳐 원하던 대학에 갈 만한 성적을 받지 못하고, 그런 선자이를 위로하던 전화 통화에서 커징텅은 불쑥 자신의 마음을 고백하는데, 막상 선자이의 대답은 듣기를 거부한 채 전화를 끊고 만다.

“정말 대답 안 듣고 싶어?”
“안 듣고 싶어. 부탁이야, 지금은 말하지 말아줘. 제발.” 나는 마음을 가라앉혔다. “조금만 기다려줘. 내가 네 남자친구가 되는 그날까지. 제발, 내가 계속 너를 좋아하게 해줘.” (본문 241쪽)

하지만 서로 다른 대학에 진학한 뒤, 그렇게 유보해둔 대답을 제대로 들을 기회가 미처 찾아오기도 전, 사랑은 미완으로 남고 만다.
활기 없이 흘러가는 대학 생활에 나른함을 느끼던 커징텅은 남학생들을 모아 격투기 대회를 열어 학우들의 호응 속에 대회를 마치는데, 직접 선수로 출전하기까지 해 몸은 흠씬 두들겨맞았으나 묘한 해방감과 희열을 맛본다. 기쁘고 들뜬 마음으로 선자이에게 전화를 건 커징텅, 하지만 커징텅의 얘기를 듣는 내내 침묵으로 일관하는 선자이. 중학교 3년, 고등학교 3년, 대학교 2년…… 총 8년 동안 둘 사이의 차이를 모르고 지내온 것은 아니었지만, 돌려 말하지 않고 직면해 내뱉은 서로의 차이점은 서로를 무척이나 아프게 했다.

“나한테 중요한 일인데 왜 나쁘게만 생각해? 이건 나라는 사람의 개성에서 중요한 한 부분이라고. 너는 이 세상에서 나를 가장 잘 이해하는 사람이면서 설마 그것도 몰라?” 나는 숨을 깊이 들이마셨다.
“너한테 가장 중요한 일이라는 게, 스스로를 다치게 하는 거란 말이야?” 선자이가 차가운 목소리로 말했다.
그 말은 아주 깊숙하고도 날카롭게 나를 찔러왔다. 웅웅 회오리치며 내 심장 깊숙이 들어온, 어찌 뽑아낼 수 없는 아픔으로, 나는 부들부들 떨었다.
“마음이 아프다.” 눈물이 났다. 더이상 화도 나지 않았다.
화를 대신해 찾아든 것은, 이해받지 못했다는 상심이었다. (본문 300쪽)

*

그렇게 두 사람은 갈림길 위에서 각자의 길을 선택하고, 얼마 지나지 않아 각각 여자친구와 남자친구를 사귄다. 하지만 둘의 이야기는 그대로 끝나지 않는다. 8년간의 설렘과 그리움은 둘 사이가 쉽게 끊어지지 않고 연결되도록 만들어주었다. 연인보다 충만하고 친구보다 충실한 그 무엇이 둘 사이에 있었다.
그리하여, 대답 듣기를 유보해두었던 고백에 대한 대답과, 미완으로 남겨진 사랑의 의미에 대한 깨달음이, 시간이 한참 흐른 뒤에야 두 사람 사이에 다시 찾아온다. 서로를 잊은 듯 지내던 어느 가을, 타이베이를 진원지로 한 지진이 일어나고, 커징텅은 신호가 잡히지 않는 휴대전화를 높이 쳐들고 달리며 애타게 선자이에게 전화가 연결되기를 기도한다.

“배터리 다돼가.”
“오늘 나한테 전화해줄 생각도 하고, 정말 고마워.”
“음. 나야말로 그때의 대답을 들려줘서 고마워. 정말 마음이 놓여. 그동안 너를 좋아한 게 나 혼자 북 치고 장구 치고 한 게 아니라 줄곧 메아리가 있었다는 걸, 알게 됐으니까. 나한테는 정말 중요하거든.” 나는 도시의 하늘을 수놓은 붉은색 별들을 바라보며 말했다. “내 청춘이, 독백이 아니었다는 거.” (본문 318쪽)


그 시절, 사랑에 의지해 성장한 우리의 청춘
― 누군가를 좋아하는 일에 정확한 시간표는 없다

선자이의 결혼식 날. 예식장에는 커징텅을 비롯해 ‘그 시절’ 선자이를 좋아했던 친구들이 모두 모였다. 수다와 일상을 즐길 줄 알고 성실한 데다가, 공부 욕심이 많은 만큼 공부를 잘하기까지 하던 선자이를 좋아하는 남학생은 커징텅 한 사람만이 아니었다. 커징텅과 거의 늘 붙어다니다시피 하며 가장 많은 장난을 함께 치고 가장 많은 시간을 함께 보낸 쉬보춘만 제외하고 랴오잉훙, 셰밍허(아허), 장자쉰, 두신셴, 양쩌위, 차오궈성 등 하나같이 제 나름의 방법으로 선자이 주변을 달처럼 빙빙 맴돌았던 것이다.

백 명의 사람이 있다면 백 가지 사랑이 있는 것이다. 그건 누군가를 좋아하는 방법이 적어도 백 가지는 존재한다는 뜻이리라. | 277쪽

‘그 시절’에 대한 소설을 일단 쓰기 시작했으나, 소설의 시작과 끝을 어떻게 열고 맺어야 할지 몰라 내내 고민중이던, 소설 속 커징텅은 선자이의 결혼식장에서 그 해답을 얻는다.

청춘이라는 이름의 파도가 밀려와 우리를 한바탕 삼켜버렸다.
파도가 물러간 뒤 온몸이 젖은 우리는 함께 모래사장에 앉아, 우리가 가장 좋아하는 소녀가 힘껏 두 손을 흔들며 인생의 다른 한쪽을 향해 행복하게 발을 내딛는 모습을 지켜보았다.
그다음 파도는, 소녀가 모래사장에 남긴 아름다운 발자국을 가지고 갔다.
그러나 우리는 아직 거기 있다.
우리 가슴속에 새겨진 소녀의 모습 또한 아직 있을 것이다.
우리는 호기로움을 잃지 않고, 그 시절을 웃으며 그린다. (본문 338쪽)

시인 서효인의 말처럼 “사실 ‘우리’가 사랑했던 건 ‘소녀’가 아닌 ‘시절’이었을지도 모른다.”

작가의 말

모두들 고향을 떠나 더욱 큰 도시에서 활짝 날개를 펼쳤습니다.

진정한 뿌리는, 땅에 있는 것이 아니라, 마음에 있는 것이라고 생각합니다.

우리 모두 계속해서 앞으로 나아가겠지만, 우리의 오랜 기억 속에 담긴 소중한 추억들은 영원히 바래지 않을 것입니다.

우리는 한자리에 모일 때마다 선자이 이야기를 합니다. 비록 과학이 발전한 덕분에, 선자이와도 모바일 메신저 앱을 통해 단체 대화방에서 매일매일 잡담을 주고받게 되어, 예전과 같은 신비로움은 사라졌지만요. 그렇다고는 해도, 어떤 기묘한 추억들은 사내 녀석들의 요란스러운 세계에만 오롯이 속한다는 것을, 여러분도 아실 거라 생각합니다. 우리는 마작을 하며, 배트를 휘두르며, 또는 온천에 몸을 담근 채, 의식하지 못한 사이 선자이의 이름을, 그 시절의 추억을 꺼냅니다.

우리들의 청춘 속에 선자이가 있다는 사실이, 정말 행복합니다.

비록 당신과 나는, 우리는 서로 만난 적은 없지만요.

2017년, 주바다오""")

때 몸 빛 사람 인 자신 거지 동안 간직 첫사랑 영화로 대만 영화 그시절우리가좋아했던소녀 동명 원작 소설 문학동네 출간 영화 그시절우리가좋아했던소녀 말 비밀 시작 소녀시대 대만 청춘 영화 계보 교복 학생 주인공 등장 청춘 꿈 희망 성장 사랑 키워드 추억 소환 시간 기억 영화 대만 청춘 영화 공통점 흥행 면 대만 영화 흥행 신호탄 말할수없는비밀 말할수없는비밀 기록 소녀시대 기록 그시절우리가좋아했던소녀 첫사랑 영화 영화 팬 존재감 인 존재감 증명 국내 처음 개봉 영화 개봉 일본 개봉 예정 리메이크 제작 영화 그시절우리가좋아했던소녀 첫사랑 서사 대표 스토리 자리 힘 각본 연출 주바 다오 감독 자전 이야기 바탕 실화 작품 중고등학교 대학 시절 주바 다오 감독 자신 청춘 기록 첫사랑 풋풋 알 상처 입체 소설 그시절우리가좋아했던소녀 잡지 연재 인 단행본 출간 작가 주바 다오 각본 감독 동명 영화 뒤 개봉 개봉 영화 대만 전체 박스오피스 기록 연속 홍콩 중화권 영화 박스오피스 자리 타이베이영화제 관객 홍콩 영화제 대만 중국 지역 최우수 영화상 중국 영화 미디어 인 감독상 작품상 수상 소녀 고개 소녀 눈망울 영화 소설 소년 징 소녀 선 자이 이야기 중심 소설 그시절우리가좋아했던소녀 영화 모태 원작 입체 영화 매체 특성 고등학교 시절 중심 축약 사건 시간 소설 원작 중학생 시절 성인 이후 생애 시간 망라 사건 심리 면 촘촘 묘사 밀도 유지 인칭 관찰자 시점 영화 징 인칭 주인공 시점 과거 현재 서술 원작 소설 주인공 징 선 자이 캐릭터 입체 인물 선택 행동 이면 심경 작품 시선 섬세 영화 원작 소설 원작 그시절우리가좋아했던소녀 자이 여학생 등장 이야기 군 오 훙 얘기 글 미국 유학중 인 이 인터넷 게시판 글 말 날개 쌍 소리 생각 고개 바람 전화 본문 오프라인 잡지 온라인 사이트 동시 다발 작품 발표 연재 활발 작품 활동 소설가 징 최근 반년 사이 어머니 병상 사색 시간 병상 곁 기억 문 어머니 자신 세밀 기록 날 청춘 단어 시작 홍수 작품 소재 고민 청춘 때 친구 조언 결정

때 몸 빛 사람 인해 자신 거지 동안 간직 첫사랑 영화로 대만 영화 그시절우리가좋아했던소녀 동명 원작 소설 문학동네 출간 영화 그시절우리가좋아했던소녀 말 비밀 시작 소녀시대 대만 청춘 영화 계보 교복 학생 주인공 등장 청춘 꿈 희망 성장 사랑 키워드 추억 소환 시간 기억 영화 대만 청춘 영화 공통점 흥행 면 대만 영화 흥행 신호탄 말할수없는비밀 말할수없는비밀 기록 소녀시대 기록 그시절우리가좋아했던소녀 첫사랑 영화 영화 팬 존재감 각인 존재감 증명 국내 처음 개봉 영화 지난 개봉 일본 개봉 예정 리메이크 제작 영화 그시절우리가좋아했던소녀 첫사랑 서사 대표 스토리 자리 힘 각본 연출 주바 다오 감독 자전 이야기 바탕 실화 작품 중고등학교 대학 시절 주바 다오 감독 자신 청춘 기록 첫사랑 풋풋 알 상처 입체 소설 그시절우리가좋아했던소녀 잡지 연재 인기 단행본 출간 작가 주바 다오 각본 감독 동명 영화 뒤 개봉 개봉 영화 대만 전체 박스오피스 기록 연속 홍콩 중화권 영화 박스오피스 자리 타이베이영화제 관객 홍콩 영화제 대만 중국 지역 최우수 영화상 중국 영화 미디어 신인 감독상 작품상 수상 소녀 고개 소녀 눈망울 영화 소설 소년 징 소녀 선 자이 이야기 중심 소설 그시절우리가좋아했던소녀 영화 모태 원작 입체 영화 매체 특성 고등학교 시절 중심 축약 사건 시간 소설 원작 중학생 시절 성인 이후 생애 시간 망라 사건 심리 면 촘촘 묘사 밀도 유지 인칭 관찰자 시점 영화 징 인칭 주인공 시점 과거 현재 서술 원작 소설 주인공 징 선 자이 캐릭터 입체 인물 선택 행동 이면 심경 작품 시선 섬세 영화 원작 소설 원작 그시절우리가좋아했던소녀 자이 여학생 등장 이야기 군 오 훙 얘기 글 미국 유학 중인 이 인터넷 게시판 글 말 날개 쌍 소리 생각 고개 바람 전화 본문 오프라인 잡지 온라인 사이트 동시 다발 작품 발표 연재 활발 작품 활동 소설가 징 최근 반년 사이 어머니 병상 사색 시간 병상 곁 기억 문 어머니 자신 세밀 기록 날 청춘 단어 시작 홍수 작품 소재 고민 청춘 때 친구 조언 결정 청춘 그녀 전화 통 계기 이야기 쓰기 시작 시절 이야기 소녀 우리 소설 속 징 시절 소설 여름 장화현 징 청 학교2 학년 미술 반 교실 시작 수업 시간 시간 소년 징 연습장 연재 만화 반 친구 게 취미 만화가 막연 믿음 성적 공부 관심 밖 일상 잔잔 파문 변화 일기 시작 수업 시간 한시 징 담임 선생님 우등 모범생 자이 앞 자리 골칫거리 부탁 말씀 자이 대답 징 일상 수다 허물 약 유치 소유자 징 노인 잔소리 고개 자이 눈망울 시선 머리 조금 공부 성적 자이 덤덤 말 이야기 시작 징 인생 교정 필요 유의 이야기 본문 잔소리 수다 자이 우등 특유 입 공부 이야기 적도 사람 마음 편안 구석 날 인가 펜 끝 느낌 자이 과목 과목 교과서 교과서 색색 형광펜 밑줄 요점 정리 메모 자이 잔소리 참견 덕분 징 성적 조금 시작 학교 반 정책 시험 성적 반 불안 속 시험 합격선 안 반 남 징 마음 생각 선생님 징 자리 배정 이번 모범생 리 샤오 뒷자리 징 수학 점수 자신 시험지 리 샤오 징 자신 해결 수학 문제 실상 자이 덕분 수학 점수 자신 수학 아이 생각 리 샤오 앞 망신 징 기 밤 수학 공부 몰두 노력 생각 사람 자신 능력 평가 노력 글자 영광 청춘 여학생 노력 끝 풍경 사실 계속 노력 세상 레드 리스트 상장 자리 머리 좋아서 리 샤오 고개 아니야 출처 자신감 자취 본문 앞뒤 자리 시간 하교길 자전거 손 망설임 가슴 리 샤오 각별 사이 동성 친구 무리 우정 징 사이에서 고민 리 샤오 친구 사이에서 소속감 대신 징 싸늘 고입 연합고사 뒤 징 장난 도모 친구 자이 징 청 중학교 울타리 징 청 고등학교 진학 리 샤오 장화 여고 선택 징 너는 머리 자연계 진학 리 샤오 한마디 관심 적성 자연계 진학 마당 추억 자락 장화 여고 진학 여학생 제외 미술 반 친구 중학교 교정 학교 담임 선생님 고등학교 바람 심상 날 심상 날 가운데 특별 자이 자이 사람 에너지 플러스마이너스 중화 상태 유지 본문 담임 선생님 벌 처음 앞뒤 징 자이 사이 에너지 한동안 덤덤 담백 유지 자이 리 샤오 속앓이 징 도움 요청 손 고등학생 날 수업 집 중학교 교실 자이 모습 사이 중화 상태 균형 의미 균열 시작 방 후 학교 공부 자이 징 공부 중학교 때 장난스레 장난스레 일상 수다 공유 장난스레 장난스레 공부 날 자이 덕분 조금 성적 성과 재미 보람 징 자이 성적 제안 시험 국 영수 과목 점수 징 자이 포니테일 머리 자이 징 머리 삭발 결과 자이 승 징 장대비 자전거 머리 교실 공부 자이 자이 모습 웃음 며칠 뒤 징 내색 포니테일 머리 자이 마음 한편 저릿하다 사랑은 사람 평소 모습 변화 사랑 매력 별것 밤낮 기도 사랑 자격 충분 사랑 본문 대학 입학 시험 날 자이 징 그림 졸업 선물 티셔츠 시험장 사과 눈동자 그림 티셔츠 징 편지 자이 시험 대학 성적 자이 위로 전화 통화 징 자신 마음 고백 자이 대답 거부 전화 정말 대답 부탁 지금 말 마음 조금 남자친구 그날 본문 대학 진학 뒤 유보 대답 기회 전 사랑은 미완 활기 대학 생활 나른 징 남학생 모아 격투기 대회 학우 호응 속 대회 선수 출전 몸 흠 해방감 희열 마음 자이 전화 징 징 얘기 침묵 일관 자이 중학교 고등학교 학교2 동안 사이 차이 말 직면 서로 차이점 서로 중요 생각 이건 나라 사람 개성 중요 부분 세상 이해 사람 숨 중요 게 스스로 거란 말 자이 목소리 말 말 깊숙 웅웅 심장 아픔 마음 눈물 이상 화도 화 대신 이해 상심 본문 사람 갈림길 위 각자 길 선택 얼마 여자친구 남자친구 이야기 간의 그리움 사이 연결 연인 충만 친구 충실 사이 대답 유보 고백 대답 미완 사랑 의미 깨달음 시간 한참 뒤 사람 사이 서로 가을 타이베이 진원지 지진 징 신호 휴대 전화 자이 전화 연결 기도 배터리 오늘 전화 생각 그때 대답 마음 그동안 게 혼자 북 장구 게 메아리 걸 중요 도시 하늘 붉은색 별 말 청춘 독백 본문 시절 사랑 의지 성장 청춘 일 정확 시간표 자이 결혼식 날 예식장 징 비롯 시절 자이 친구 다와 일상 성실 공부 욕심 공부 자이 남학생 징 사람 징 장난 시간 만 제외 오 훙 셰밍허 아허 장자 두신셴 양쩌위 차오궈성 방법 자이 주변 달 사람 사랑 방법 존재 뜻 시절 소설 쓰기 시작 소설 시작 끝 고민 소설 속 징 자이 결혼식장 해답 청춘 이름 파도 파도 뒤 온몸 모래사장 소녀 손 인생 한쪽 행복 발 모습 다음 파도 소녀 모래사장 발자국 가슴속 소녀 모습 호기 시절 본문 시인 서효인 말 사실 우리 사랑 소녀 시절 작가 말 모두 고향 도시 날개 진정 뿌리 땅 마음 생각 계속 앞 기억 속 소중 추억 한자리 때 자이 이야기 과학 발전 덕분 자이 와도 모바일 메신저 앱 단체 대화방 잡담 예전 신비 기묘 추억 사내 요란 세계 속 생각 마작 배트 온천 몸 의식 사이 자이 이름 시절 추억 우리들의 청춘 속 자이 사실 행복합니다 주바 다오

# 새 모델 테스트

In [29]:
# komoran_2 = Komoran(modelpath='./komoran/', userdic="./datasets/user_dic.txt")

In [30]:
# komoran_2.nouns("테스트")